In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# 10 fold cross validation for fine-tuning"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "def lee_filter(img, size):\n",
    "    \n",
    "    '''\n",
    "    Remove noise and sparkling in the images\n",
    "    \n",
    "    return new image with less noise\n",
    "    '''\n",
    "    \n",
    "    img_mean = uniform_filter(img, (size, size))\n",
    "    img_sqr_mean = uniform_filter(img**2, (size, size))\n",
    "    img_variance = img_sqr_mean - img_mean**2\n",
    "\n",
    "    overall_variance = variance(img)\n",
    "\n",
    "    img_weights = img_variance**2 / (img_variance**2 + overall_variance**2)\n",
    "    img_output = img_mean + img_weights * (img - img_mean)\n",
    "\n",
    "    return img_output"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 11,
   "metadata": {},
   "outputs": [],
   "source": [
    "def get_scaled_imgs(df):\n",
    "    \n",
    "    '''\n",
    "    Read the image from excel file and convert it into an images with colour composite\n",
    "    \n",
    "    return an array of images\n",
    "    '''\n",
    "    \n",
    "    \n",
    "    imgs = []\n",
    "    \n",
    "    for i, row in df.iterrows():\n",
    "        #make 75x75 image\n",
    "        band_1 = np.array(row['band_1']).reshape(75, 75)\n",
    "        band_2 = np.array(row['band_2']).reshape(75, 75)\n",
    "        band_3 = (band_1 + band_2)/2 # plus since log(x*y) = log(x) + log(y)\n",
    "        \n",
    "        # use a lee filter to help with speckling\n",
    "        band_1 = lee_filter(band_1,4)\n",
    "        band_2 = lee_filter(band_2,4)\n",
    "        band_3 = lee_filter(band_3,4)\n",
    "        \n",
    "        # Rescale\n",
    "        a = (band_1 - band_1.mean()) / (band_1.max() - band_1.min())\n",
    "        b = (band_2 - band_2.mean()) / (band_2.max() - band_2.min())\n",
    "        c = (band_3 - band_3.mean()) / (band_3.max() - band_3.min())\n",
    "\n",
    "        imgs.append(np.dstack((a, b, c)))\n",
    "#        imgs.append(np.dstack((a)))\n",
    "\n",
    "    return np.array(imgs)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 12,
   "metadata": {},
   "outputs": [],
   "source": [
    "import cv2\n",
    "def get_more_images(imgs):\n",
    "    \n",
    "    '''\n",
    "    Image augmentation to increase the size of the original images\n",
    "    by using horizontal flip and vertical flip\n",
    "    \n",
    "    return new augmented images is append to the train dataset.\n",
    "    \n",
    "    '''\n",
    "    \n",
    "    more_images = []\n",
    "    vert_flip_imgs = []\n",
    "    hori_flip_imgs = []\n",
    "      \n",
    "    for i in range(0,imgs.shape[0]):\n",
    "        a=imgs[i,:,:,0]\n",
    "        b=imgs[i,:,:,1]\n",
    "        c=imgs[i,:,:,2]\n",
    "        \n",
    "        av=cv2.flip(a,1)\n",
    "        ah=cv2.flip(a,0)\n",
    "        bv=cv2.flip(b,1)\n",
    "        bh=cv2.flip(b,0)\n",
    "        cv=cv2.flip(c,1)\n",
    "        ch=cv2.flip(c,0)\n",
    "        \n",
    "        vert_flip_imgs.append(np.dstack((av, bv, cv)))\n",
    "        hori_flip_imgs.append(np.dstack((ah, bh, ch)))\n",
    "      \n",
    "    v = np.array(vert_flip_imgs)\n",
    "    h = np.array(hori_flip_imgs)\n",
    "       \n",
    "    more_images = np.concatenate((imgs,v,h))\n",
    "    \n",
    "    return more_images"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 13,
   "metadata": {},
   "outputs": [
    {
     "name": "stderr",
     "output_type": "stream",
     "text": [
      "/home/paperspace/anaconda3/envs/fastai/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.\n",
      "  from ._conv import register_converters as _register_converters\n",
      "Using TensorFlow backend.\n"
     ]
    }
   ],
   "source": [
    "#import required modules for training\n",
    "\n",
    "import os\n",
    "import numpy as np\n",
    "import pandas as pd\n",
    "\n",
    "import keras\n",
    "from keras.preprocessing.image import ImageDataGenerator\n",
    "from keras.preprocessing.image import *\n",
    "from keras.layers import GlobalMaxPooling2D\n",
    "from keras.layers.normalization import BatchNormalization\n",
    "from keras.layers.merge import Concatenate\n",
    "from keras import initializers\n",
    "from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping, ReduceLROnPlateau\n",
    "\n",
    "from scipy.ndimage.filters import uniform_filter\n",
    "from scipy.ndimage.measurements import variance\n",
    "\n",
    "from keras.models import Sequential, Model\n",
    "from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Input, Flatten, Add, GlobalMaxPooling2D\n",
    "from keras.layers import Concatenate, concatenate\n",
    "from keras.layers import Activation, LeakyReLU\n",
    "from keras.optimizers import Adam, RMSprop, SGD, Adadelta\n",
    "from sklearn.model_selection import train_test_split, StratifiedKFold\n",
    "from sklearn.metrics import accuracy_score\n",
    "\n",
    "seed = 1234\n",
    "np.random.seed(seed)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 14,
   "metadata": {},
   "outputs": [],
   "source": [
    "train = pd.read_json('train.json')\n",
    "\n",
    "X_train = get_scaled_imgs(train)\n",
    "y_train = np.array(train['is_iceberg'])\n",
    "\n",
    "train.inc_angle = train.inc_angle.replace('na',0)\n",
    "idx_tr = np.where(train.inc_angle>0)\n",
    "\n",
    "y_train = y_train[idx_tr[0]]\n",
    "X_train = X_train[idx_tr[0],...]\n",
    "\n",
    "#X_train = get_more_images(X_train) \n",
    "#y_train = np.concatenate((y_train,y_train,y_train))\n",
    "\n",
    "X_train, X_test, y_train, y_test = train_test_split( X_train, y_train, test_size=0.1, random_state=42)\n",
    "\n",
    "folds = list(StratifiedKFold(n_splits=10, shuffle=True, random_state=1).split(X_train, y_train))"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 6,
   "metadata": {},
   "outputs": [],
   "source": [
    "def get_model():\n",
    "    \n",
    "    \"\"\"\n",
    "    Keras Sequential model\n",
    "\n",
    "    \"\"\"\n",
    "    model=Sequential()\n",
    "    \n",
    "    # Conv block 1\n",
    "    model.add(Conv2D(94, kernel_size=(3, 3), input_shape=(75, 75, 3)))\n",
    "    model.add(Activation('relu'))\n",
    "    model.add(Conv2D(94, kernel_size=(3, 3)))\n",
    "    model.add(Activation('relu'))\n",
    "    model.add(Conv2D(94, kernel_size=(3, 3)))\n",
    "    model.add(Activation('relu'))\n",
    "    model.add(BatchNormalization())\n",
    "    model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))\n",
    "    model.add(Dropout(0.25))\n",
    "   \n",
    "    # Conv block 2\n",
    "    model.add(Conv2D(192, kernel_size=(3, 3)))\n",
    "    model.add(Activation('relu'))\n",
    "    model.add(Conv2D(192, kernel_size=(3, 3)))\n",
    "    model.add(Activation('relu'))\n",
    "    model.add(Conv2D(192, kernel_size=(3, 3)))\n",
    "    model.add(Activation('relu'))\n",
    "    model.add(BatchNormalization())\n",
    "    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))\n",
    "    model.add(Dropout(0.25))\n",
    "    \n",
    "    # Conv block 3\n",
    "    model.add(Conv2D(192, kernel_size=(3, 3)))\n",
    "    model.add(BatchNormalization())\n",
    "    model.add(Activation('relu'))\n",
    "    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))\n",
    "    model.add(Dropout(0.25))\n",
    "\n",
    "    # Flatten before dense\n",
    "    model.add(Flatten())\n",
    "\n",
    "    #Dense 1\n",
    "    model.add(Dense(1024, activation='relu'))\n",
    "    model.add(Dropout(0.4))\n",
    "\n",
    "    #Dense 2\n",
    "    model.add(Dense(512, activation='relu'))\n",
    "    model.add(Dropout(0.2))\n",
    "\n",
    "    # Output \n",
    "    model.add(Dense(1, activation=\"sigmoid\"))\n",
    "\n",
    "    optimizer = Adam(lr=0.0001, decay=0.0)\n",
    "    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])\n",
    "    \n",
    "    return model"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 7,
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "_________________________________________________________________\n",
      "Layer (type)                 Output Shape              Param #   \n",
      "=================================================================\n",
      "conv2d_1 (Conv2D)            (None, 73, 73, 94)        2632      \n",
      "_________________________________________________________________\n",
      "activation_1 (Activation)    (None, 73, 73, 94)        0         \n",
      "_________________________________________________________________\n",
      "conv2d_2 (Conv2D)            (None, 71, 71, 94)        79618     \n",
      "_________________________________________________________________\n",
      "activation_2 (Activation)    (None, 71, 71, 94)        0         \n",
      "_________________________________________________________________\n",
      "conv2d_3 (Conv2D)            (None, 69, 69, 94)        79618     \n",
      "_________________________________________________________________\n",
      "activation_3 (Activation)    (None, 69, 69, 94)        0         \n",
      "_________________________________________________________________\n",
      "batch_normalization_1 (Batch (None, 69, 69, 94)        376       \n",
      "_________________________________________________________________\n",
      "max_pooling2d_1 (MaxPooling2 (None, 34, 34, 94)        0         \n",
      "_________________________________________________________________\n",
      "dropout_1 (Dropout)          (None, 34, 34, 94)        0         \n",
      "_________________________________________________________________\n",
      "conv2d_4 (Conv2D)            (None, 32, 32, 192)       162624    \n",
      "_________________________________________________________________\n",
      "activation_4 (Activation)    (None, 32, 32, 192)       0         \n",
      "_________________________________________________________________\n",
      "conv2d_5 (Conv2D)            (None, 30, 30, 192)       331968    \n",
      "_________________________________________________________________\n",
      "activation_5 (Activation)    (None, 30, 30, 192)       0         \n",
      "_________________________________________________________________\n",
      "conv2d_6 (Conv2D)            (None, 28, 28, 192)       331968    \n",
      "_________________________________________________________________\n",
      "activation_6 (Activation)    (None, 28, 28, 192)       0         \n",
      "_________________________________________________________________\n",
      "batch_normalization_2 (Batch (None, 28, 28, 192)       768       \n",
      "_________________________________________________________________\n",
      "max_pooling2d_2 (MaxPooling2 (None, 14, 14, 192)       0         \n",
      "_________________________________________________________________\n",
      "dropout_2 (Dropout)          (None, 14, 14, 192)       0         \n",
      "_________________________________________________________________\n",
      "conv2d_7 (Conv2D)            (None, 12, 12, 192)       331968    \n",
      "_________________________________________________________________\n",
      "batch_normalization_3 (Batch (None, 12, 12, 192)       768       \n",
      "_________________________________________________________________\n",
      "activation_7 (Activation)    (None, 12, 12, 192)       0         \n",
      "_________________________________________________________________\n",
      "max_pooling2d_3 (MaxPooling2 (None, 6, 6, 192)         0         \n",
      "_________________________________________________________________\n",
      "dropout_3 (Dropout)          (None, 6, 6, 192)         0         \n",
      "_________________________________________________________________\n",
      "flatten_1 (Flatten)          (None, 6912)              0         \n",
      "_________________________________________________________________\n",
      "dense_1 (Dense)              (None, 1024)              7078912   \n",
      "_________________________________________________________________\n",
      "dropout_4 (Dropout)          (None, 1024)              0         \n",
      "_________________________________________________________________\n",
      "dense_2 (Dense)              (None, 512)               524800    \n",
      "_________________________________________________________________\n",
      "dropout_5 (Dropout)          (None, 512)               0         \n",
      "_________________________________________________________________\n",
      "dense_3 (Dense)              (None, 1)                 513       \n",
      "=================================================================\n",
      "Total params: 8,926,533\n",
      "Trainable params: 8,925,577\n",
      "Non-trainable params: 956\n",
      "_________________________________________________________________\n"
     ]
    }
   ],
   "source": [
    "#summary of the CNN model\n",
    "model = get_model()\n",
    "model.summary()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 8,
   "metadata": {},
   "outputs": [],
   "source": [
    "#defined parameter\n",
    "batch_size=32\n",
    "\n",
    "#real-time image augmentation\n",
    "gen = ImageDataGenerator(\n",
    "                         zoom_range = 0.25,\n",
    "                        )"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 9,
   "metadata": {},
   "outputs": [],
   "source": [
    "#save best model only and create an earlyStopping function once the model reach a maximum accuracy\n",
    "def get_callbacks(name_weights, patience_lr):\n",
    "    mcp_save = ModelCheckpoint(name_weights, save_best_only=True, monitor='val_loss', mode='min')\n",
    "    reduce_lr_loss = ReduceLROnPlateau(monitor='loss', factor=0.1, patience=patience_lr, verbose=1, epsilon=1e-4, mode='min')\n",
    "    earlyStopping = EarlyStopping(monitor='val_loss', patience=15, verbose=0, mode='min')\n",
    "    return [mcp_save, reduce_lr_loss, earlyStopping]"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 10,
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "\n",
      "Fold  0\n",
      "Epoch 1/30\n",
      "38/37 [==============================] - 8s 198ms/step - loss: 0.8069 - acc: 0.5816 - val_loss: 0.5747 - val_acc: 0.6791\n",
      "Epoch 2/30\n",
      "38/37 [==============================] - 4s 115ms/step - loss: 0.5911 - acc: 0.7208 - val_loss: 0.4611 - val_acc: 0.7612\n",
      "Epoch 3/30\n",
      "38/37 [==============================] - 4s 114ms/step - loss: 0.5106 - acc: 0.7404 - val_loss: 0.3367 - val_acc: 0.8358\n",
      "Epoch 4/30\n",
      "38/37 [==============================] - 4s 114ms/step - loss: 0.3933 - acc: 0.8011 - val_loss: 0.4919 - val_acc: 0.7463\n",
      "Epoch 5/30\n",
      "38/37 [==============================] - 4s 115ms/step - loss: 0.3754 - acc: 0.8202 - val_loss: 0.5006 - val_acc: 0.7687\n",
      "Epoch 6/30\n",
      "38/37 [==============================] - 4s 114ms/step - loss: 0.4416 - acc: 0.8037 - val_loss: 0.2734 - val_acc: 0.8731\n",
      "Epoch 7/30\n",
      "38/37 [==============================] - 4s 115ms/step - loss: 0.4037 - acc: 0.8223 - val_loss: 0.2820 - val_acc: 0.8806\n",
      "Epoch 8/30\n",
      "38/37 [==============================] - 4s 115ms/step - loss: 0.2934 - acc: 0.8749 - val_loss: 0.2284 - val_acc: 0.8806\n",
      "Epoch 9/30\n",
      "38/37 [==============================] - 4s 115ms/step - loss: 0.2792 - acc: 0.8708 - val_loss: 0.2622 - val_acc: 0.8806\n",
      "Epoch 10/30\n",
      "38/37 [==============================] - 4s 115ms/step - loss: 0.2925 - acc: 0.8731 - val_loss: 0.3391 - val_acc: 0.8358\n",
      "Epoch 11/30\n",
      "38/37 [==============================] - 4s 115ms/step - loss: 0.3316 - acc: 0.8498 - val_loss: 0.3047 - val_acc: 0.8582\n",
      "Epoch 12/30\n",
      "38/37 [==============================] - 4s 115ms/step - loss: 0.2777 - acc: 0.8766 - val_loss: 0.3844 - val_acc: 0.8209\n",
      "Epoch 13/30\n",
      "38/37 [==============================] - 4s 115ms/step - loss: 0.2496 - acc: 0.8936 - val_loss: 0.3002 - val_acc: 0.8507\n",
      "Epoch 14/30\n",
      "38/37 [==============================] - 4s 116ms/step - loss: 0.2631 - acc: 0.8868 - val_loss: 0.2817 - val_acc: 0.8731\n",
      "Epoch 15/30\n",
      "38/37 [==============================] - 4s 115ms/step - loss: 0.2407 - acc: 0.8969 - val_loss: 0.6696 - val_acc: 0.7463\n",
      "Epoch 16/30\n",
      "38/37 [==============================] - 5s 119ms/step - loss: 0.2706 - acc: 0.8873 - val_loss: 0.2834 - val_acc: 0.8806\n",
      "Epoch 17/30\n",
      "38/37 [==============================] - 4s 115ms/step - loss: 0.2364 - acc: 0.8963 - val_loss: 0.1836 - val_acc: 0.9328\n",
      "Epoch 18/30\n",
      "38/37 [==============================] - 4s 117ms/step - loss: 0.2037 - acc: 0.9202 - val_loss: 0.1963 - val_acc: 0.9254\n",
      "Epoch 19/30\n",
      "38/37 [==============================] - 4s 116ms/step - loss: 0.2305 - acc: 0.9035 - val_loss: 0.2539 - val_acc: 0.8955\n",
      "Epoch 20/30\n",
      "38/37 [==============================] - 4s 116ms/step - loss: 0.2416 - acc: 0.9043 - val_loss: 0.2453 - val_acc: 0.9104\n",
      "Epoch 21/30\n",
      "38/37 [==============================] - 4s 116ms/step - loss: 0.2243 - acc: 0.9126 - val_loss: 0.7279 - val_acc: 0.7463\n",
      "Epoch 22/30\n",
      "38/37 [==============================] - 4s 117ms/step - loss: 0.2048 - acc: 0.9082 - val_loss: 0.5776 - val_acc: 0.7910\n",
      "Epoch 23/30\n",
      "38/37 [==============================] - 4s 116ms/step - loss: 0.2147 - acc: 0.9134 - val_loss: 0.9982 - val_acc: 0.7164\n",
      "Epoch 24/30\n",
      "38/37 [==============================] - 4s 116ms/step - loss: 0.2588 - acc: 0.8953 - val_loss: 0.5161 - val_acc: 0.7836\n",
      "Epoch 25/30\n",
      "38/37 [==============================] - 4s 116ms/step - loss: 0.2084 - acc: 0.9152 - val_loss: 0.2314 - val_acc: 0.9179\n",
      "Epoch 26/30\n",
      "38/37 [==============================] - 4s 115ms/step - loss: 0.1652 - acc: 0.9358 - val_loss: 0.4078 - val_acc: 0.8507\n",
      "Epoch 27/30\n",
      "38/37 [==============================] - 4s 116ms/step - loss: 0.1660 - acc: 0.9375 - val_loss: 0.3165 - val_acc: 0.8731\n",
      "Epoch 28/30\n",
      "38/37 [==============================] - 4s 116ms/step - loss: 0.1640 - acc: 0.9284 - val_loss: 0.4295 - val_acc: 0.8507\n",
      "Epoch 29/30\n",
      "38/37 [==============================] - 4s 116ms/step - loss: 0.1560 - acc: 0.9307 - val_loss: 0.3130 - val_acc: 0.8657\n",
      "Epoch 30/30\n",
      "38/37 [==============================] - 4s 116ms/step - loss: 0.1936 - acc: 0.9235 - val_loss: 0.4193 - val_acc: 0.8507\n",
      "134/134 [==============================] - 0s 1ms/step\n",
      "\n",
      " Val score: 0.18362756006753267\n",
      "\n",
      " Val accuracy: 0.9328358200059008\n",
      "         id  is_iceberg\n",
      "0  d4bb8bac    0.947607\n",
      "1  28612c22    0.000104\n",
      "2  d5c8ab27    0.000031\n",
      "3  b8cf5edb    0.664554\n",
      "4  3f6778c5    0.453289\n",
      "5  0ecf7d79    0.000148\n",
      "6  4acfe35a    0.832597\n",
      "7  a5b4d551    0.000048\n",
      "8  cacbe718    0.997896\n",
      "9  de1244d8    0.955124\n",
      "submission saved\n",
      "\n",
      "Fold  1\n",
      "Epoch 1/30\n",
      "38/37 [==============================] - 5s 143ms/step - loss: 0.7756 - acc: 0.6221 - val_loss: 0.5289 - val_acc: 0.6842\n",
      "Epoch 2/30\n",
      "38/37 [==============================] - 4s 116ms/step - loss: 0.5703 - acc: 0.7433 - val_loss: 0.5912 - val_acc: 0.6992\n",
      "Epoch 3/30\n",
      "38/37 [==============================] - 5s 119ms/step - loss: 0.5087 - acc: 0.7678 - val_loss: 0.4320 - val_acc: 0.8045\n",
      "Epoch 4/30\n",
      "38/37 [==============================] - 4s 116ms/step - loss: 0.4319 - acc: 0.7875 - val_loss: 0.5396 - val_acc: 0.7820\n",
      "Epoch 5/30\n",
      "38/37 [==============================] - 4s 116ms/step - loss: 0.3854 - acc: 0.8254 - val_loss: 0.3942 - val_acc: 0.8120\n",
      "Epoch 6/30\n",
      "38/37 [==============================] - 4s 116ms/step - loss: 0.3395 - acc: 0.8517 - val_loss: 0.6819 - val_acc: 0.7218\n",
      "Epoch 7/30\n",
      "38/37 [==============================] - 4s 116ms/step - loss: 0.3434 - acc: 0.8318 - val_loss: 0.3355 - val_acc: 0.8797\n",
      "Epoch 8/30\n",
      "38/37 [==============================] - 4s 116ms/step - loss: 0.2874 - acc: 0.8690 - val_loss: 0.3602 - val_acc: 0.8647\n",
      "Epoch 9/30\n",
      "38/37 [==============================] - 4s 116ms/step - loss: 0.2915 - acc: 0.8779 - val_loss: 0.3494 - val_acc: 0.8346\n",
      "Epoch 10/30\n",
      "38/37 [==============================] - 4s 116ms/step - loss: 0.2523 - acc: 0.8881 - val_loss: 0.4072 - val_acc: 0.8271\n",
      "Epoch 11/30\n",
      "38/37 [==============================] - 4s 116ms/step - loss: 0.2654 - acc: 0.8747 - val_loss: 0.3628 - val_acc: 0.8346\n",
      "Epoch 12/30\n",
      "38/37 [==============================] - 4s 116ms/step - loss: 0.2646 - acc: 0.8889 - val_loss: 1.4105 - val_acc: 0.6090\n",
      "Epoch 13/30\n",
      "38/37 [==============================] - 4s 116ms/step - loss: 0.2706 - acc: 0.8887 - val_loss: 0.3728 - val_acc: 0.8271\n",
      "Epoch 14/30\n",
      "38/37 [==============================] - 4s 116ms/step - loss: 0.2420 - acc: 0.9087 - val_loss: 0.4359 - val_acc: 0.8271\n",
      "Epoch 15/30\n",
      "38/37 [==============================] - 4s 118ms/step - loss: 0.2151 - acc: 0.9019 - val_loss: 0.3182 - val_acc: 0.8571\n",
      "Epoch 16/30\n",
      "38/37 [==============================] - 4s 116ms/step - loss: 0.2038 - acc: 0.9175 - val_loss: 0.4687 - val_acc: 0.8120\n",
      "Epoch 17/30\n",
      "38/37 [==============================] - 4s 116ms/step - loss: 0.2999 - acc: 0.8721 - val_loss: 0.3631 - val_acc: 0.8421\n",
      "Epoch 18/30\n",
      "38/37 [==============================] - 4s 117ms/step - loss: 0.1852 - acc: 0.9101 - val_loss: 0.5561 - val_acc: 0.7820\n",
      "Epoch 19/30\n",
      "38/37 [==============================] - 4s 116ms/step - loss: 0.2045 - acc: 0.9116 - val_loss: 0.4596 - val_acc: 0.8496\n",
      "Epoch 20/30\n",
      "38/37 [==============================] - 4s 116ms/step - loss: 0.1900 - acc: 0.9235 - val_loss: 0.4107 - val_acc: 0.8271\n",
      "Epoch 21/30\n",
      "38/37 [==============================] - 4s 116ms/step - loss: 0.2677 - acc: 0.9017 - val_loss: 0.3135 - val_acc: 0.8797\n",
      "Epoch 22/30\n",
      "38/37 [==============================] - 4s 116ms/step - loss: 0.1807 - acc: 0.9276 - val_loss: 0.7244 - val_acc: 0.7895\n",
      "Epoch 23/30\n",
      "38/37 [==============================] - 4s 116ms/step - loss: 0.1862 - acc: 0.9301 - val_loss: 0.4533 - val_acc: 0.8571\n",
      "Epoch 24/30\n",
      "38/37 [==============================] - 4s 116ms/step - loss: 0.1748 - acc: 0.9292 - val_loss: 0.4219 - val_acc: 0.8421\n",
      "Epoch 25/30\n",
      "38/37 [==============================] - 4s 116ms/step - loss: 0.1742 - acc: 0.9276 - val_loss: 0.4465 - val_acc: 0.8271\n",
      "Epoch 26/30\n",
      "38/37 [==============================] - 4s 117ms/step - loss: 0.2253 - acc: 0.9067 - val_loss: 0.6094 - val_acc: 0.8271\n",
      "Epoch 27/30\n",
      "38/37 [==============================] - 4s 116ms/step - loss: 0.1713 - acc: 0.9299 - val_loss: 0.4194 - val_acc: 0.8496\n",
      "Epoch 28/30\n",
      "38/37 [==============================] - 4s 116ms/step - loss: 0.1521 - acc: 0.9383 - val_loss: 0.4253 - val_acc: 0.8421\n",
      "Epoch 29/30\n",
      "38/37 [==============================] - 4s 116ms/step - loss: 0.1442 - acc: 0.9440 - val_loss: 0.7525 - val_acc: 0.7669\n",
      "Epoch 30/30\n",
      "38/37 [==============================] - 4s 116ms/step - loss: 0.1677 - acc: 0.9340 - val_loss: 0.4468 - val_acc: 0.8120\n",
      "133/133 [==============================] - 0s 1ms/step\n",
      "\n",
      " Val score: 0.31347784237529996\n",
      "\n",
      " Val accuracy: 0.8796992481203008\n",
      "         id  is_iceberg\n",
      "0  d4bb8bac    0.988759\n",
      "1  28612c22    0.000002\n",
      "2  d5c8ab27    0.000678\n",
      "3  b8cf5edb    0.515260\n",
      "4  3f6778c5    0.617451\n",
      "5  0ecf7d79    0.000112\n",
      "6  4acfe35a    0.919109\n",
      "7  a5b4d551    0.000012\n",
      "8  cacbe718    0.997888\n",
      "9  de1244d8    0.993062\n",
      "submission saved\n",
      "\n",
      "Fold  2\n",
      "Epoch 1/30\n",
      "38/37 [==============================] - 6s 157ms/step - loss: 0.8435 - acc: 0.6022 - val_loss: 0.5203 - val_acc: 0.7500\n",
      "Epoch 2/30\n",
      "38/37 [==============================] - 4s 116ms/step - loss: 0.5843 - acc: 0.7063 - val_loss: 0.3881 - val_acc: 0.8561\n",
      "Epoch 3/30\n",
      "38/37 [==============================] - 4s 116ms/step - loss: 0.5243 - acc: 0.7515 - val_loss: 0.3174 - val_acc: 0.8788\n",
      "Epoch 4/30\n",
      "38/37 [==============================] - 4s 116ms/step - loss: 0.4469 - acc: 0.7828 - val_loss: 0.4534 - val_acc: 0.7576\n",
      "Epoch 5/30\n",
      "38/37 [==============================] - 4s 117ms/step - loss: 0.4071 - acc: 0.8046 - val_loss: 0.4596 - val_acc: 0.7576\n",
      "Epoch 6/30\n",
      "38/37 [==============================] - 4s 116ms/step - loss: 0.3375 - acc: 0.8453 - val_loss: 0.2897 - val_acc: 0.8561\n",
      "Epoch 7/30\n",
      "38/37 [==============================] - 4s 116ms/step - loss: 0.3607 - acc: 0.8272 - val_loss: 0.3743 - val_acc: 0.7803\n",
      "Epoch 8/30\n",
      "38/37 [==============================] - 4s 116ms/step - loss: 0.3541 - acc: 0.8388 - val_loss: 1.1684 - val_acc: 0.5606\n",
      "Epoch 9/30\n",
      "38/37 [==============================] - 4s 116ms/step - loss: 0.3116 - acc: 0.8540 - val_loss: 0.2662 - val_acc: 0.8636\n",
      "Epoch 10/30\n",
      "38/37 [==============================] - 4s 117ms/step - loss: 0.3109 - acc: 0.8609 - val_loss: 0.2502 - val_acc: 0.8864\n",
      "Epoch 11/30\n",
      "38/37 [==============================] - 4s 116ms/step - loss: 0.2820 - acc: 0.8688 - val_loss: 0.2518 - val_acc: 0.8636\n",
      "Epoch 12/30\n",
      "38/37 [==============================] - 4s 116ms/step - loss: 0.2771 - acc: 0.8692 - val_loss: 0.2696 - val_acc: 0.8485\n",
      "Epoch 13/30\n",
      "38/37 [==============================] - 4s 117ms/step - loss: 0.2562 - acc: 0.8865 - val_loss: 0.1802 - val_acc: 0.9470\n",
      "Epoch 14/30\n",
      "38/37 [==============================] - 4s 116ms/step - loss: 0.2280 - acc: 0.9009 - val_loss: 0.2182 - val_acc: 0.8712\n",
      "Epoch 15/30\n",
      "38/37 [==============================] - 4s 116ms/step - loss: 0.2321 - acc: 0.9000 - val_loss: 0.2887 - val_acc: 0.8712\n",
      "Epoch 16/30\n",
      "38/37 [==============================] - 4s 116ms/step - loss: 0.2385 - acc: 0.9066 - val_loss: 1.1691 - val_acc: 0.5985\n",
      "Epoch 17/30\n",
      "38/37 [==============================] - 4s 116ms/step - loss: 0.2209 - acc: 0.9025 - val_loss: 0.2253 - val_acc: 0.8788\n",
      "Epoch 18/30\n",
      "38/37 [==============================] - 4s 116ms/step - loss: 0.2652 - acc: 0.8857 - val_loss: 0.1942 - val_acc: 0.9167\n",
      "Epoch 19/30\n",
      "38/37 [==============================] - 4s 117ms/step - loss: 0.1981 - acc: 0.9194 - val_loss: 0.2828 - val_acc: 0.8636\n",
      "Epoch 20/30\n",
      "38/37 [==============================] - 4s 116ms/step - loss: 0.2238 - acc: 0.9029 - val_loss: 0.9098 - val_acc: 0.6818\n",
      "Epoch 21/30\n",
      "38/37 [==============================] - 4s 116ms/step - loss: 0.2386 - acc: 0.9009 - val_loss: 0.4394 - val_acc: 0.7879\n",
      "Epoch 22/30\n",
      "38/37 [==============================] - 4s 116ms/step - loss: 0.2015 - acc: 0.9120 - val_loss: 0.3980 - val_acc: 0.8106\n",
      "Epoch 23/30\n",
      "38/37 [==============================] - 4s 116ms/step - loss: 0.1726 - acc: 0.9350 - val_loss: 0.2241 - val_acc: 0.8788\n",
      "Epoch 24/30\n",
      "38/37 [==============================] - 4s 116ms/step - loss: 0.1706 - acc: 0.9321 - val_loss: 0.1869 - val_acc: 0.9091\n",
      "Epoch 25/30\n",
      "38/37 [==============================] - 4s 116ms/step - loss: 0.1664 - acc: 0.9334 - val_loss: 1.7276 - val_acc: 0.6288\n",
      "Epoch 26/30\n",
      "38/37 [==============================] - 4s 116ms/step - loss: 0.2043 - acc: 0.9153 - val_loss: 0.2024 - val_acc: 0.9318\n",
      "Epoch 27/30\n",
      "38/37 [==============================] - 4s 116ms/step - loss: 0.1523 - acc: 0.9391 - val_loss: 0.2205 - val_acc: 0.8939\n",
      "Epoch 28/30\n",
      "38/37 [==============================] - 4s 116ms/step - loss: 0.1786 - acc: 0.9223 - val_loss: 0.2075 - val_acc: 0.8939\n",
      "132/132 [==============================] - 0s 1ms/step\n",
      "\n",
      " Val score: 0.1801526700005387\n",
      "\n",
      " Val accuracy: 0.946969696969697\n",
      "         id  is_iceberg\n",
      "0  d4bb8bac    0.964356\n",
      "1  28612c22    0.000031\n",
      "2  d5c8ab27    0.000477\n",
      "3  b8cf5edb    0.230417\n",
      "4  3f6778c5    0.372624\n",
      "5  0ecf7d79    0.000055\n",
      "6  4acfe35a    0.667340\n",
      "7  a5b4d551    0.000010\n",
      "8  cacbe718    0.980856\n",
      "9  de1244d8    0.892555\n",
      "submission saved\n",
      "\n",
      "Fold  3\n",
      "Epoch 1/30\n",
      "38/37 [==============================] - 6s 145ms/step - loss: 0.8666 - acc: 0.5619 - val_loss: 0.7574 - val_acc: 0.5833\n",
      "Epoch 2/30\n",
      "38/37 [==============================] - 4s 116ms/step - loss: 0.5597 - acc: 0.7174 - val_loss: 0.6345 - val_acc: 0.6364\n",
      "Epoch 3/30\n",
      "38/37 [==============================] - 4s 117ms/step - loss: 0.5092 - acc: 0.7549 - val_loss: 0.3772 - val_acc: 0.8636\n",
      "Epoch 4/30\n",
      "38/37 [==============================] - 4s 117ms/step - loss: 0.4469 - acc: 0.7910 - val_loss: 0.5435 - val_acc: 0.7576\n",
      "Epoch 5/30\n",
      "38/37 [==============================] - 4s 116ms/step - loss: 0.3996 - acc: 0.8190 - val_loss: 0.3615 - val_acc: 0.8182\n",
      "Epoch 6/30\n",
      "38/37 [==============================] - 4s 116ms/step - loss: 0.3202 - acc: 0.8478 - val_loss: 0.2910 - val_acc: 0.8409\n",
      "Epoch 7/30\n",
      "38/37 [==============================] - 4s 116ms/step - loss: 0.2882 - acc: 0.8651 - val_loss: 0.7767 - val_acc: 0.6970\n",
      "Epoch 8/30\n",
      "38/37 [==============================] - 4s 117ms/step - loss: 0.3369 - acc: 0.8486 - val_loss: 0.5651 - val_acc: 0.7576\n",
      "Epoch 9/30\n",
      "38/37 [==============================] - 4s 116ms/step - loss: 0.2734 - acc: 0.8815 - val_loss: 0.4695 - val_acc: 0.8258\n",
      "Epoch 10/30\n",
      "38/37 [==============================] - 4s 116ms/step - loss: 0.2967 - acc: 0.8688 - val_loss: 0.5262 - val_acc: 0.7652\n",
      "Epoch 11/30\n",
      "38/37 [==============================] - 4s 116ms/step - loss: 0.2454 - acc: 0.8927 - val_loss: 0.4287 - val_acc: 0.8409\n",
      "Epoch 12/30\n",
      "38/37 [==============================] - 4s 116ms/step - loss: 0.3091 - acc: 0.8556 - val_loss: 0.4434 - val_acc: 0.7955\n",
      "Epoch 13/30\n",
      "38/37 [==============================] - 4s 117ms/step - loss: 0.2716 - acc: 0.8869 - val_loss: 0.4793 - val_acc: 0.7879\n",
      "Epoch 14/30\n",
      "38/37 [==============================] - 4s 117ms/step - loss: 0.2116 - acc: 0.9054 - val_loss: 0.4033 - val_acc: 0.8788\n",
      "Epoch 15/30\n",
      "38/37 [==============================] - 4s 116ms/step - loss: 0.2237 - acc: 0.9054 - val_loss: 0.2893 - val_acc: 0.8788\n",
      "Epoch 16/30\n",
      "38/37 [==============================] - 4s 116ms/step - loss: 0.2307 - acc: 0.9021 - val_loss: 0.2889 - val_acc: 0.9015\n",
      "Epoch 17/30\n",
      "38/37 [==============================] - 4s 116ms/step - loss: 0.2209 - acc: 0.9046 - val_loss: 0.6660 - val_acc: 0.7348\n",
      "Epoch 18/30\n",
      "38/37 [==============================] - 4s 116ms/step - loss: 0.2222 - acc: 0.8906 - val_loss: 0.2027 - val_acc: 0.9091\n",
      "Epoch 19/30\n",
      "38/37 [==============================] - 4s 116ms/step - loss: 0.2098 - acc: 0.9194 - val_loss: 0.3085 - val_acc: 0.8712\n",
      "Epoch 20/30\n",
      "38/37 [==============================] - 4s 116ms/step - loss: 0.2087 - acc: 0.9120 - val_loss: 0.2470 - val_acc: 0.9167\n",
      "Epoch 21/30\n",
      "38/37 [==============================] - 4s 116ms/step - loss: 0.1835 - acc: 0.9259 - val_loss: 0.7096 - val_acc: 0.7879\n",
      "Epoch 22/30\n",
      "38/37 [==============================] - 4s 116ms/step - loss: 0.2349 - acc: 0.8980 - val_loss: 0.2557 - val_acc: 0.8864\n",
      "Epoch 23/30\n",
      "38/37 [==============================] - 4s 116ms/step - loss: 0.1921 - acc: 0.9038 - val_loss: 0.2561 - val_acc: 0.9167\n",
      "Epoch 24/30\n",
      "38/37 [==============================] - 4s 116ms/step - loss: 0.1618 - acc: 0.9309 - val_loss: 0.3183 - val_acc: 0.9015\n",
      "Epoch 25/30\n",
      "38/37 [==============================] - 4s 116ms/step - loss: 0.1770 - acc: 0.9239 - val_loss: 0.2369 - val_acc: 0.9242\n",
      "Epoch 26/30\n"
     ]
    },
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "38/37 [==============================] - 4s 116ms/step - loss: 0.1565 - acc: 0.9363 - val_loss: 0.3150 - val_acc: 0.8939\n",
      "Epoch 27/30\n",
      "38/37 [==============================] - 4s 117ms/step - loss: 0.1572 - acc: 0.9391 - val_loss: 0.3944 - val_acc: 0.8712\n",
      "Epoch 28/30\n",
      "38/37 [==============================] - 4s 116ms/step - loss: 0.2162 - acc: 0.9075 - val_loss: 0.3692 - val_acc: 0.8561\n",
      "Epoch 29/30\n",
      "38/37 [==============================] - 4s 117ms/step - loss: 0.1677 - acc: 0.9292 - val_loss: 0.3360 - val_acc: 0.8788\n",
      "Epoch 30/30\n",
      "38/37 [==============================] - 4s 116ms/step - loss: 0.1504 - acc: 0.9395 - val_loss: 0.2659 - val_acc: 0.9015\n",
      "132/132 [==============================] - 0s 1ms/step\n",
      "\n",
      " Val score: 0.20271667950984204\n",
      "\n",
      " Val accuracy: 0.9090909090909091\n",
      "         id    is_iceberg\n",
      "0  d4bb8bac  9.194921e-01\n",
      "1  28612c22  6.721845e-08\n",
      "2  d5c8ab27  8.144771e-06\n",
      "3  b8cf5edb  1.431399e-01\n",
      "4  3f6778c5  1.799062e-01\n",
      "5  0ecf7d79  8.762927e-06\n",
      "6  4acfe35a  4.634774e-01\n",
      "7  a5b4d551  3.153087e-08\n",
      "8  cacbe718  9.878531e-01\n",
      "9  de1244d8  9.590520e-01\n",
      "submission saved\n",
      "\n",
      "Fold  4\n",
      "Epoch 1/30\n",
      "38/37 [==============================] - 6s 159ms/step - loss: 0.7800 - acc: 0.6108 - val_loss: 0.7129 - val_acc: 0.6439\n",
      "Epoch 2/30\n",
      "38/37 [==============================] - 4s 116ms/step - loss: 0.6157 - acc: 0.6993 - val_loss: 0.6787 - val_acc: 0.6742\n",
      "Epoch 3/30\n",
      "38/37 [==============================] - 4s 117ms/step - loss: 0.4963 - acc: 0.7598 - val_loss: 0.9473 - val_acc: 0.6515\n",
      "Epoch 4/30\n",
      "38/37 [==============================] - 4s 116ms/step - loss: 0.4746 - acc: 0.7725 - val_loss: 0.3857 - val_acc: 0.7879\n",
      "Epoch 5/30\n",
      "38/37 [==============================] - 4s 116ms/step - loss: 0.3889 - acc: 0.8145 - val_loss: 0.3683 - val_acc: 0.8333\n",
      "Epoch 6/30\n",
      "38/37 [==============================] - 4s 116ms/step - loss: 0.3243 - acc: 0.8495 - val_loss: 0.3549 - val_acc: 0.8182\n",
      "Epoch 7/30\n",
      "38/37 [==============================] - 4s 116ms/step - loss: 0.3212 - acc: 0.8573 - val_loss: 0.3021 - val_acc: 0.8561\n",
      "Epoch 8/30\n",
      "38/37 [==============================] - 4s 117ms/step - loss: 0.3000 - acc: 0.8675 - val_loss: 0.3738 - val_acc: 0.8409\n",
      "Epoch 9/30\n",
      "38/37 [==============================] - 4s 117ms/step - loss: 0.2525 - acc: 0.8930 - val_loss: 0.5242 - val_acc: 0.7879\n",
      "Epoch 10/30\n",
      "38/37 [==============================] - 4s 116ms/step - loss: 0.2859 - acc: 0.8729 - val_loss: 0.2984 - val_acc: 0.8864\n",
      "Epoch 11/30\n",
      "38/37 [==============================] - 4s 116ms/step - loss: 0.2802 - acc: 0.8836 - val_loss: 0.8672 - val_acc: 0.6970\n",
      "Epoch 12/30\n",
      "38/37 [==============================] - 4s 116ms/step - loss: 0.3118 - acc: 0.8643 - val_loss: 0.2980 - val_acc: 0.8712\n",
      "Epoch 13/30\n",
      "38/37 [==============================] - 4s 116ms/step - loss: 0.2320 - acc: 0.9070 - val_loss: 0.6134 - val_acc: 0.7727\n",
      "Epoch 14/30\n",
      "38/37 [==============================] - 4s 116ms/step - loss: 0.2084 - acc: 0.9136 - val_loss: 0.5402 - val_acc: 0.7803\n",
      "Epoch 15/30\n",
      "38/37 [==============================] - 4s 116ms/step - loss: 0.2089 - acc: 0.9070 - val_loss: 0.2170 - val_acc: 0.9242\n",
      "Epoch 16/30\n",
      "38/37 [==============================] - 4s 117ms/step - loss: 0.2229 - acc: 0.9021 - val_loss: 0.3104 - val_acc: 0.8561\n",
      "Epoch 17/30\n",
      "38/37 [==============================] - 4s 117ms/step - loss: 0.2236 - acc: 0.9091 - val_loss: 0.2127 - val_acc: 0.9091\n",
      "Epoch 18/30\n",
      "38/37 [==============================] - 4s 116ms/step - loss: 0.2096 - acc: 0.9087 - val_loss: 0.4467 - val_acc: 0.8333\n",
      "Epoch 19/30\n",
      "38/37 [==============================] - 4s 116ms/step - loss: 0.2220 - acc: 0.9103 - val_loss: 0.3304 - val_acc: 0.8636\n",
      "Epoch 20/30\n",
      "38/37 [==============================] - 4s 117ms/step - loss: 0.1848 - acc: 0.9136 - val_loss: 0.6702 - val_acc: 0.7652\n",
      "Epoch 21/30\n",
      "38/37 [==============================] - 4s 116ms/step - loss: 0.1990 - acc: 0.9185 - val_loss: 0.2941 - val_acc: 0.8409\n",
      "Epoch 22/30\n",
      "38/37 [==============================] - 4s 116ms/step - loss: 0.1972 - acc: 0.9214 - val_loss: 0.4361 - val_acc: 0.8333\n",
      "Epoch 23/30\n",
      "38/37 [==============================] - 4s 116ms/step - loss: 0.2039 - acc: 0.9181 - val_loss: 0.2275 - val_acc: 0.9015\n",
      "Epoch 24/30\n",
      "38/37 [==============================] - 4s 116ms/step - loss: 0.1666 - acc: 0.9428 - val_loss: 0.2602 - val_acc: 0.9091\n",
      "Epoch 25/30\n",
      "38/37 [==============================] - 4s 117ms/step - loss: 0.1809 - acc: 0.9214 - val_loss: 0.4018 - val_acc: 0.8561\n",
      "Epoch 26/30\n",
      "38/37 [==============================] - 4s 117ms/step - loss: 0.1856 - acc: 0.9288 - val_loss: 0.2300 - val_acc: 0.8939\n",
      "Epoch 27/30\n",
      "38/37 [==============================] - 4s 116ms/step - loss: 0.1804 - acc: 0.9321 - val_loss: 0.5217 - val_acc: 0.8258\n",
      "Epoch 28/30\n",
      "38/37 [==============================] - 4s 116ms/step - loss: 0.1761 - acc: 0.9223 - val_loss: 0.2189 - val_acc: 0.9091\n",
      "Epoch 29/30\n",
      "38/37 [==============================] - 4s 116ms/step - loss: 0.1747 - acc: 0.9280 - val_loss: 0.2281 - val_acc: 0.8939\n",
      "Epoch 30/30\n",
      "38/37 [==============================] - 4s 116ms/step - loss: 0.1680 - acc: 0.9387 - val_loss: 0.2056 - val_acc: 0.9015\n",
      "132/132 [==============================] - 0s 1ms/step\n",
      "\n",
      " Val score: 0.20561417666348544\n",
      "\n",
      " Val accuracy: 0.9015151515151515\n",
      "         id    is_iceberg\n",
      "0  d4bb8bac  8.858589e-01\n",
      "1  28612c22  6.408218e-06\n",
      "2  d5c8ab27  6.499324e-04\n",
      "3  b8cf5edb  5.106408e-01\n",
      "4  3f6778c5  6.008700e-01\n",
      "5  0ecf7d79  3.829202e-09\n",
      "6  4acfe35a  9.457577e-01\n",
      "7  a5b4d551  6.207761e-06\n",
      "8  cacbe718  9.982456e-01\n",
      "9  de1244d8  9.961877e-01\n",
      "submission saved\n",
      "\n",
      "Fold  5\n",
      "Epoch 1/30\n",
      "38/37 [==============================] - 6s 154ms/step - loss: 0.8473 - acc: 0.5833 - val_loss: 0.5681 - val_acc: 0.7197\n",
      "Epoch 2/30\n",
      "38/37 [==============================] - 4s 117ms/step - loss: 0.5905 - acc: 0.7104 - val_loss: 0.5718 - val_acc: 0.6894\n",
      "Epoch 3/30\n",
      "38/37 [==============================] - 4s 117ms/step - loss: 0.5267 - acc: 0.7524 - val_loss: 0.3953 - val_acc: 0.8258\n",
      "Epoch 4/30\n",
      "38/37 [==============================] - 4s 117ms/step - loss: 0.4631 - acc: 0.7824 - val_loss: 0.4562 - val_acc: 0.7879\n",
      "Epoch 5/30\n",
      "38/37 [==============================] - 4s 117ms/step - loss: 0.4206 - acc: 0.7972 - val_loss: 0.5038 - val_acc: 0.7803\n",
      "Epoch 6/30\n",
      "38/37 [==============================] - 4s 116ms/step - loss: 0.3731 - acc: 0.8149 - val_loss: 0.8690 - val_acc: 0.6288\n",
      "Epoch 7/30\n",
      "38/37 [==============================] - 4s 116ms/step - loss: 0.2912 - acc: 0.8634 - val_loss: 0.2771 - val_acc: 0.8485\n",
      "Epoch 8/30\n",
      "38/37 [==============================] - 4s 116ms/step - loss: 0.3296 - acc: 0.8487 - val_loss: 0.5050 - val_acc: 0.7576\n",
      "Epoch 9/30\n",
      "38/37 [==============================] - 4s 116ms/step - loss: 0.3306 - acc: 0.8478 - val_loss: 0.3280 - val_acc: 0.8182\n",
      "Epoch 10/30\n",
      "38/37 [==============================] - 4s 117ms/step - loss: 0.2601 - acc: 0.8836 - val_loss: 0.2701 - val_acc: 0.8864\n",
      "Epoch 11/30\n",
      "38/37 [==============================] - 4s 116ms/step - loss: 0.2488 - acc: 0.8844 - val_loss: 0.3849 - val_acc: 0.8485\n",
      "Epoch 12/30\n",
      "38/37 [==============================] - 4s 117ms/step - loss: 0.2362 - acc: 0.8951 - val_loss: 0.2322 - val_acc: 0.8636\n",
      "Epoch 13/30\n",
      "38/37 [==============================] - 4s 116ms/step - loss: 0.2565 - acc: 0.8922 - val_loss: 0.4226 - val_acc: 0.8030\n",
      "Epoch 14/30\n",
      "38/37 [==============================] - 4s 116ms/step - loss: 0.2294 - acc: 0.9042 - val_loss: 0.3362 - val_acc: 0.8561\n",
      "Epoch 15/30\n",
      "38/37 [==============================] - 4s 116ms/step - loss: 0.2011 - acc: 0.9177 - val_loss: 0.2178 - val_acc: 0.9015\n",
      "Epoch 16/30\n",
      "38/37 [==============================] - 4s 117ms/step - loss: 0.2372 - acc: 0.9128 - val_loss: 0.3212 - val_acc: 0.8485\n",
      "Epoch 17/30\n",
      "38/37 [==============================] - 4s 117ms/step - loss: 0.2186 - acc: 0.9095 - val_loss: 0.3583 - val_acc: 0.8485\n",
      "Epoch 18/30\n",
      "38/37 [==============================] - 4s 116ms/step - loss: 0.2221 - acc: 0.9058 - val_loss: 0.2352 - val_acc: 0.8788\n",
      "Epoch 19/30\n",
      "38/37 [==============================] - 4s 117ms/step - loss: 0.2047 - acc: 0.9202 - val_loss: 0.1932 - val_acc: 0.9091\n",
      "Epoch 20/30\n",
      "38/37 [==============================] - 4s 117ms/step - loss: 0.2004 - acc: 0.9181 - val_loss: 0.2215 - val_acc: 0.9091\n",
      "Epoch 21/30\n",
      "38/37 [==============================] - 4s 116ms/step - loss: 0.2454 - acc: 0.8935 - val_loss: 0.2099 - val_acc: 0.8939\n",
      "Epoch 22/30\n",
      "38/37 [==============================] - 4s 117ms/step - loss: 0.1914 - acc: 0.9206 - val_loss: 0.2031 - val_acc: 0.9015\n",
      "Epoch 23/30\n",
      "38/37 [==============================] - 4s 117ms/step - loss: 0.1977 - acc: 0.9185 - val_loss: 0.2292 - val_acc: 0.8939\n",
      "Epoch 24/30\n",
      "38/37 [==============================] - 4s 117ms/step - loss: 0.1955 - acc: 0.9227 - val_loss: 0.2294 - val_acc: 0.8864\n",
      "Epoch 25/30\n",
      "38/37 [==============================] - 4s 118ms/step - loss: 0.1708 - acc: 0.9309 - val_loss: 0.3417 - val_acc: 0.8485\n",
      "Epoch 26/30\n",
      "38/37 [==============================] - 4s 117ms/step - loss: 0.1721 - acc: 0.9346 - val_loss: 0.2476 - val_acc: 0.9015\n",
      "Epoch 27/30\n",
      "38/37 [==============================] - 4s 117ms/step - loss: 0.1721 - acc: 0.9223 - val_loss: 0.2354 - val_acc: 0.8939\n",
      "Epoch 28/30\n",
      "38/37 [==============================] - 4s 117ms/step - loss: 0.1879 - acc: 0.9284 - val_loss: 0.5948 - val_acc: 0.7803\n",
      "Epoch 29/30\n",
      "38/37 [==============================] - 4s 117ms/step - loss: 0.1691 - acc: 0.9309 - val_loss: 0.2505 - val_acc: 0.8939\n",
      "Epoch 30/30\n",
      "38/37 [==============================] - 4s 117ms/step - loss: 0.1802 - acc: 0.9190 - val_loss: 0.2475 - val_acc: 0.8788\n",
      "132/132 [==============================] - 0s 1ms/step\n",
      "\n",
      " Val score: 0.19322156996437997\n",
      "\n",
      " Val accuracy: 0.9090909090909091\n",
      "         id    is_iceberg\n",
      "0  d4bb8bac  9.736684e-01\n",
      "1  28612c22  5.501763e-06\n",
      "2  d5c8ab27  1.979638e-03\n",
      "3  b8cf5edb  2.881460e-01\n",
      "4  3f6778c5  3.600430e-01\n",
      "5  0ecf7d79  5.195190e-08\n",
      "6  4acfe35a  8.787867e-01\n",
      "7  a5b4d551  1.720600e-07\n",
      "8  cacbe718  9.909629e-01\n",
      "9  de1244d8  9.638814e-01\n",
      "submission saved\n",
      "\n",
      "Fold  6\n",
      "Epoch 1/30\n",
      "38/37 [==============================] - 6s 160ms/step - loss: 0.8334 - acc: 0.6101 - val_loss: 0.6946 - val_acc: 0.6212\n",
      "Epoch 2/30\n",
      "38/37 [==============================] - 4s 116ms/step - loss: 0.5855 - acc: 0.7153 - val_loss: 0.6448 - val_acc: 0.6515\n",
      "Epoch 3/30\n",
      "38/37 [==============================] - 4s 116ms/step - loss: 0.5166 - acc: 0.7450 - val_loss: 0.5838 - val_acc: 0.7197\n",
      "Epoch 4/30\n",
      "38/37 [==============================] - 4s 117ms/step - loss: 0.4278 - acc: 0.8140 - val_loss: 0.6095 - val_acc: 0.7045\n",
      "Epoch 5/30\n",
      "38/37 [==============================] - 4s 117ms/step - loss: 0.3944 - acc: 0.8211 - val_loss: 0.3696 - val_acc: 0.8333\n",
      "Epoch 6/30\n",
      "38/37 [==============================] - 4s 117ms/step - loss: 0.3358 - acc: 0.8474 - val_loss: 0.8402 - val_acc: 0.6742\n",
      "Epoch 7/30\n",
      "38/37 [==============================] - 4s 116ms/step - loss: 0.3487 - acc: 0.8486 - val_loss: 0.3560 - val_acc: 0.8106\n",
      "Epoch 8/30\n",
      "38/37 [==============================] - 4s 117ms/step - loss: 0.2986 - acc: 0.8626 - val_loss: 0.4389 - val_acc: 0.8182\n",
      "Epoch 9/30\n",
      "38/37 [==============================] - 4s 116ms/step - loss: 0.2639 - acc: 0.8840 - val_loss: 0.7247 - val_acc: 0.7273\n",
      "Epoch 10/30\n",
      "38/37 [==============================] - 4s 117ms/step - loss: 0.2762 - acc: 0.8865 - val_loss: 0.3993 - val_acc: 0.7424\n",
      "Epoch 11/30\n",
      "38/37 [==============================] - 4s 116ms/step - loss: 0.2321 - acc: 0.8931 - val_loss: 0.3411 - val_acc: 0.8333\n",
      "Epoch 12/30\n",
      "38/37 [==============================] - 4s 116ms/step - loss: 0.2858 - acc: 0.8778 - val_loss: 0.7630 - val_acc: 0.6818\n",
      "Epoch 13/30\n",
      "38/37 [==============================] - 4s 116ms/step - loss: 0.2455 - acc: 0.9037 - val_loss: 0.3098 - val_acc: 0.8712\n",
      "Epoch 14/30\n",
      "38/37 [==============================] - 4s 116ms/step - loss: 0.2259 - acc: 0.9013 - val_loss: 0.2860 - val_acc: 0.8636\n",
      "Epoch 15/30\n",
      "38/37 [==============================] - 4s 116ms/step - loss: 0.2520 - acc: 0.8996 - val_loss: 1.1685 - val_acc: 0.6742\n",
      "Epoch 16/30\n",
      "38/37 [==============================] - 4s 116ms/step - loss: 0.2085 - acc: 0.9099 - val_loss: 0.3325 - val_acc: 0.8712\n",
      "Epoch 17/30\n",
      "38/37 [==============================] - 4s 116ms/step - loss: 0.2028 - acc: 0.9144 - val_loss: 0.3556 - val_acc: 0.8561\n",
      "Epoch 18/30\n",
      "38/37 [==============================] - 4s 116ms/step - loss: 0.1983 - acc: 0.9120 - val_loss: 0.9098 - val_acc: 0.7424\n",
      "Epoch 19/30\n",
      "38/37 [==============================] - 4s 116ms/step - loss: 0.1865 - acc: 0.9227 - val_loss: 0.5026 - val_acc: 0.8182\n",
      "Epoch 20/30\n",
      "38/37 [==============================] - 4s 116ms/step - loss: 0.1846 - acc: 0.9202 - val_loss: 0.9437 - val_acc: 0.6894\n",
      "Epoch 21/30\n",
      "38/37 [==============================] - 4s 116ms/step - loss: 0.2139 - acc: 0.9144 - val_loss: 0.3564 - val_acc: 0.8409\n",
      "Epoch 22/30\n",
      "38/37 [==============================] - 4s 116ms/step - loss: 0.1592 - acc: 0.9342 - val_loss: 0.5026 - val_acc: 0.8485\n",
      "Epoch 23/30\n",
      "38/37 [==============================] - 4s 116ms/step - loss: 0.1892 - acc: 0.9181 - val_loss: 0.2668 - val_acc: 0.8712\n",
      "Epoch 24/30\n",
      "38/37 [==============================] - 4s 117ms/step - loss: 0.1933 - acc: 0.9173 - val_loss: 0.2599 - val_acc: 0.8864\n",
      "Epoch 25/30\n",
      "38/37 [==============================] - 4s 117ms/step - loss: 0.1755 - acc: 0.9313 - val_loss: 0.9109 - val_acc: 0.7500\n",
      "Epoch 26/30\n",
      "38/37 [==============================] - 4s 116ms/step - loss: 0.1835 - acc: 0.9178 - val_loss: 0.2514 - val_acc: 0.8712\n",
      "Epoch 27/30\n",
      "38/37 [==============================] - 4s 116ms/step - loss: 0.1598 - acc: 0.9334 - val_loss: 0.3498 - val_acc: 0.8636\n",
      "Epoch 28/30\n",
      "38/37 [==============================] - 4s 116ms/step - loss: 0.1499 - acc: 0.9375 - val_loss: 0.2678 - val_acc: 0.8864\n",
      "Epoch 29/30\n",
      "38/37 [==============================] - 4s 116ms/step - loss: 0.1632 - acc: 0.9383 - val_loss: 0.5903 - val_acc: 0.7955\n",
      "Epoch 30/30\n",
      "38/37 [==============================] - 4s 117ms/step - loss: 0.1352 - acc: 0.9416 - val_loss: 0.3523 - val_acc: 0.8561\n",
      "132/132 [==============================] - 0s 1ms/step\n",
      "\n",
      " Val score: 0.25144492157480935\n",
      "\n",
      " Val accuracy: 0.8712121212121212\n",
      "         id    is_iceberg\n",
      "0  d4bb8bac  9.897760e-01\n",
      "1  28612c22  3.768134e-06\n",
      "2  d5c8ab27  1.984584e-02\n",
      "3  b8cf5edb  4.914321e-01\n",
      "4  3f6778c5  3.564688e-01\n",
      "5  0ecf7d79  6.066945e-07\n",
      "6  4acfe35a  9.647377e-01\n",
      "7  a5b4d551  7.014306e-06\n",
      "8  cacbe718  9.961113e-01\n",
      "9  de1244d8  9.855646e-01\n",
      "submission saved\n",
      "\n",
      "Fold  7\n",
      "Epoch 1/30\n",
      "38/37 [==============================] - 6s 165ms/step - loss: 0.8381 - acc: 0.6150 - val_loss: 0.6025 - val_acc: 0.6591\n",
      "Epoch 2/30\n",
      "38/37 [==============================] - 4s 116ms/step - loss: 0.5558 - acc: 0.7277 - val_loss: 0.4711 - val_acc: 0.8030\n",
      "Epoch 3/30\n",
      "38/37 [==============================] - 4s 116ms/step - loss: 0.5394 - acc: 0.7561 - val_loss: 0.4376 - val_acc: 0.7652\n",
      "Epoch 4/30\n",
      "38/37 [==============================] - 4s 117ms/step - loss: 0.4410 - acc: 0.7840 - val_loss: 0.3140 - val_acc: 0.8182\n",
      "Epoch 5/30\n",
      "38/37 [==============================] - 4s 117ms/step - loss: 0.3894 - acc: 0.8140 - val_loss: 0.4435 - val_acc: 0.7500\n",
      "Epoch 6/30\n",
      "38/37 [==============================] - 4s 116ms/step - loss: 0.3571 - acc: 0.8326 - val_loss: 0.3761 - val_acc: 0.7955\n",
      "Epoch 7/30\n",
      "38/37 [==============================] - 4s 117ms/step - loss: 0.3133 - acc: 0.8601 - val_loss: 0.2431 - val_acc: 0.8864\n",
      "Epoch 8/30\n",
      "38/37 [==============================] - 4s 117ms/step - loss: 0.3052 - acc: 0.8634 - val_loss: 0.2659 - val_acc: 0.8636\n",
      "Epoch 9/30\n",
      "38/37 [==============================] - 4s 118ms/step - loss: 0.3286 - acc: 0.8610 - val_loss: 0.1736 - val_acc: 0.9318\n",
      "Epoch 10/30\n",
      "38/37 [==============================] - 4s 117ms/step - loss: 0.2700 - acc: 0.8861 - val_loss: 0.2318 - val_acc: 0.9015\n",
      "Epoch 11/30\n",
      "38/37 [==============================] - 4s 117ms/step - loss: 0.2344 - acc: 0.9004 - val_loss: 1.2864 - val_acc: 0.6136\n",
      "Epoch 12/30\n",
      "38/37 [==============================] - 4s 117ms/step - loss: 0.2588 - acc: 0.8943 - val_loss: 0.1711 - val_acc: 0.9242\n",
      "Epoch 13/30\n",
      "38/37 [==============================] - 4s 117ms/step - loss: 0.2473 - acc: 0.8918 - val_loss: 0.1198 - val_acc: 0.9470\n",
      "Epoch 14/30\n",
      "38/37 [==============================] - 4s 118ms/step - loss: 0.2100 - acc: 0.9111 - val_loss: 0.1528 - val_acc: 0.9394\n",
      "Epoch 15/30\n",
      "38/37 [==============================] - 4s 118ms/step - loss: 0.2263 - acc: 0.9050 - val_loss: 0.1198 - val_acc: 0.9470\n",
      "Epoch 16/30\n"
     ]
    },
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "38/37 [==============================] - 4s 117ms/step - loss: 0.2284 - acc: 0.9004 - val_loss: 0.6575 - val_acc: 0.7500\n",
      "Epoch 17/30\n",
      "38/37 [==============================] - 4s 118ms/step - loss: 0.2230 - acc: 0.8980 - val_loss: 0.3499 - val_acc: 0.8106\n",
      "Epoch 18/30\n",
      "38/37 [==============================] - 4s 117ms/step - loss: 0.2169 - acc: 0.9083 - val_loss: 0.3858 - val_acc: 0.8561\n",
      "Epoch 19/30\n",
      "38/37 [==============================] - 4s 117ms/step - loss: 0.2019 - acc: 0.9185 - val_loss: 0.1376 - val_acc: 0.9470\n",
      "Epoch 20/30\n",
      "38/37 [==============================] - 4s 118ms/step - loss: 0.1841 - acc: 0.9268 - val_loss: 0.4879 - val_acc: 0.8333\n",
      "Epoch 21/30\n",
      "38/37 [==============================] - 4s 118ms/step - loss: 0.2121 - acc: 0.9108 - val_loss: 0.2324 - val_acc: 0.9015\n",
      "Epoch 22/30\n",
      "38/37 [==============================] - 4s 117ms/step - loss: 0.2083 - acc: 0.9124 - val_loss: 0.1217 - val_acc: 0.9545\n",
      "Epoch 23/30\n",
      "38/37 [==============================] - 4s 117ms/step - loss: 0.2025 - acc: 0.9239 - val_loss: 0.4338 - val_acc: 0.8333\n",
      "Epoch 24/30\n",
      "38/37 [==============================] - 4s 117ms/step - loss: 0.1698 - acc: 0.9383 - val_loss: 0.1082 - val_acc: 0.9545\n",
      "Epoch 25/30\n",
      "38/37 [==============================] - 4s 116ms/step - loss: 0.1718 - acc: 0.9292 - val_loss: 0.1356 - val_acc: 0.9470\n",
      "Epoch 26/30\n",
      "38/37 [==============================] - 4s 117ms/step - loss: 0.2042 - acc: 0.9099 - val_loss: 0.1717 - val_acc: 0.9242\n",
      "Epoch 27/30\n",
      "38/37 [==============================] - 4s 117ms/step - loss: 0.1629 - acc: 0.9342 - val_loss: 0.1539 - val_acc: 0.9318\n",
      "Epoch 28/30\n",
      "38/37 [==============================] - 4s 117ms/step - loss: 0.1700 - acc: 0.9317 - val_loss: 0.1329 - val_acc: 0.9394\n",
      "Epoch 29/30\n",
      "38/37 [==============================] - 4s 116ms/step - loss: 0.1389 - acc: 0.9449 - val_loss: 0.4879 - val_acc: 0.8258\n",
      "Epoch 30/30\n",
      "38/37 [==============================] - 4s 116ms/step - loss: 0.1759 - acc: 0.9284 - val_loss: 0.4769 - val_acc: 0.8409\n",
      "132/132 [==============================] - 0s 1ms/step\n",
      "\n",
      " Val score: 0.10822843630431277\n",
      "\n",
      " Val accuracy: 0.9545454545454546\n",
      "         id    is_iceberg\n",
      "0  d4bb8bac  9.441718e-01\n",
      "1  28612c22  6.585086e-08\n",
      "2  d5c8ab27  1.033631e-04\n",
      "3  b8cf5edb  6.893286e-03\n",
      "4  3f6778c5  1.513270e-01\n",
      "5  0ecf7d79  5.546181e-08\n",
      "6  4acfe35a  3.308537e-01\n",
      "7  a5b4d551  1.483844e-08\n",
      "8  cacbe718  9.937755e-01\n",
      "9  de1244d8  9.793635e-01\n",
      "submission saved\n",
      "\n",
      "Fold  8\n",
      "Epoch 1/30\n",
      "38/37 [==============================] - 6s 170ms/step - loss: 0.8382 - acc: 0.5886 - val_loss: 0.4893 - val_acc: 0.8106\n",
      "Epoch 2/30\n",
      "38/37 [==============================] - 4s 117ms/step - loss: 0.6697 - acc: 0.6565 - val_loss: 0.4892 - val_acc: 0.7803\n",
      "Epoch 3/30\n",
      "38/37 [==============================] - 4s 116ms/step - loss: 0.4895 - acc: 0.7643 - val_loss: 0.5071 - val_acc: 0.8030\n",
      "Epoch 4/30\n",
      "38/37 [==============================] - 4s 117ms/step - loss: 0.4228 - acc: 0.7972 - val_loss: 0.3539 - val_acc: 0.8182\n",
      "Epoch 5/30\n",
      "38/37 [==============================] - 4s 117ms/step - loss: 0.3591 - acc: 0.8326 - val_loss: 0.3888 - val_acc: 0.8409\n",
      "Epoch 6/30\n",
      "38/37 [==============================] - 4s 117ms/step - loss: 0.3560 - acc: 0.8268 - val_loss: 0.4001 - val_acc: 0.8333\n",
      "Epoch 7/30\n",
      "38/37 [==============================] - 4s 117ms/step - loss: 0.3071 - acc: 0.8573 - val_loss: 0.3509 - val_acc: 0.8333\n",
      "Epoch 8/30\n",
      "38/37 [==============================] - 4s 117ms/step - loss: 0.3171 - acc: 0.8565 - val_loss: 0.4349 - val_acc: 0.8258\n",
      "Epoch 9/30\n",
      "38/37 [==============================] - 4s 117ms/step - loss: 0.2609 - acc: 0.8774 - val_loss: 0.4637 - val_acc: 0.7955\n",
      "Epoch 10/30\n",
      "38/37 [==============================] - 4s 117ms/step - loss: 0.2590 - acc: 0.8897 - val_loss: 0.2849 - val_acc: 0.8864\n",
      "Epoch 11/30\n",
      "38/37 [==============================] - 4s 116ms/step - loss: 0.2548 - acc: 0.8971 - val_loss: 0.3898 - val_acc: 0.8636\n",
      "Epoch 12/30\n",
      "38/37 [==============================] - 4s 117ms/step - loss: 0.2611 - acc: 0.8881 - val_loss: 0.2232 - val_acc: 0.8939\n",
      "Epoch 13/30\n",
      "38/37 [==============================] - 4s 117ms/step - loss: 0.2422 - acc: 0.9009 - val_loss: 0.2734 - val_acc: 0.8864\n",
      "Epoch 14/30\n",
      "38/37 [==============================] - 4s 117ms/step - loss: 0.2766 - acc: 0.8819 - val_loss: 0.3284 - val_acc: 0.8788\n",
      "Epoch 15/30\n",
      "38/37 [==============================] - 4s 116ms/step - loss: 0.2723 - acc: 0.8890 - val_loss: 0.3211 - val_acc: 0.8788\n",
      "Epoch 16/30\n",
      "38/37 [==============================] - 4s 117ms/step - loss: 0.2720 - acc: 0.8783 - val_loss: 0.2881 - val_acc: 0.9167\n",
      "Epoch 17/30\n",
      "38/37 [==============================] - 4s 117ms/step - loss: 0.2324 - acc: 0.8971 - val_loss: 0.1946 - val_acc: 0.9394\n",
      "Epoch 18/30\n",
      "38/37 [==============================] - 4s 117ms/step - loss: 0.2129 - acc: 0.9169 - val_loss: 0.2048 - val_acc: 0.9167\n",
      "Epoch 19/30\n",
      "38/37 [==============================] - 4s 117ms/step - loss: 0.1935 - acc: 0.9128 - val_loss: 0.2028 - val_acc: 0.9015\n",
      "Epoch 20/30\n",
      "38/37 [==============================] - 4s 116ms/step - loss: 0.2040 - acc: 0.9116 - val_loss: 0.5710 - val_acc: 0.8258\n",
      "Epoch 21/30\n",
      "38/37 [==============================] - 4s 117ms/step - loss: 0.1916 - acc: 0.9284 - val_loss: 0.7030 - val_acc: 0.7879\n",
      "Epoch 22/30\n",
      "38/37 [==============================] - 4s 117ms/step - loss: 0.2059 - acc: 0.9124 - val_loss: 0.2279 - val_acc: 0.9167\n",
      "Epoch 23/30\n",
      "38/37 [==============================] - 4s 117ms/step - loss: 0.1764 - acc: 0.9284 - val_loss: 0.3593 - val_acc: 0.8712\n",
      "Epoch 24/30\n",
      "38/37 [==============================] - 4s 116ms/step - loss: 0.1706 - acc: 0.9202 - val_loss: 0.4517 - val_acc: 0.8485\n",
      "Epoch 25/30\n",
      "38/37 [==============================] - 4s 116ms/step - loss: 0.1939 - acc: 0.9202 - val_loss: 0.2754 - val_acc: 0.8864\n",
      "Epoch 26/30\n",
      "38/37 [==============================] - 4s 116ms/step - loss: 0.1682 - acc: 0.9309 - val_loss: 0.2647 - val_acc: 0.9015\n",
      "Epoch 27/30\n",
      "38/37 [==============================] - 4s 117ms/step - loss: 0.1436 - acc: 0.9408 - val_loss: 0.2562 - val_acc: 0.9167\n",
      "Epoch 28/30\n",
      "38/37 [==============================] - 4s 117ms/step - loss: 0.1754 - acc: 0.9288 - val_loss: 0.6084 - val_acc: 0.8258\n",
      "Epoch 29/30\n",
      "38/37 [==============================] - 4s 117ms/step - loss: 0.1274 - acc: 0.9465 - val_loss: 0.2756 - val_acc: 0.9167\n",
      "Epoch 30/30\n",
      "38/37 [==============================] - 4s 117ms/step - loss: 0.1642 - acc: 0.9371 - val_loss: 0.8393 - val_acc: 0.7500\n",
      "132/132 [==============================] - 0s 1ms/step\n",
      "\n",
      " Val score: 0.19457238950681957\n",
      "\n",
      " Val accuracy: 0.9393939393939394\n",
      "         id  is_iceberg\n",
      "0  d4bb8bac    0.930919\n",
      "1  28612c22    0.000057\n",
      "2  d5c8ab27    0.000110\n",
      "3  b8cf5edb    0.302515\n",
      "4  3f6778c5    0.534991\n",
      "5  0ecf7d79    0.000003\n",
      "6  4acfe35a    0.890523\n",
      "7  a5b4d551    0.000020\n",
      "8  cacbe718    0.997324\n",
      "9  de1244d8    0.974539\n",
      "submission saved\n",
      "\n",
      "Fold  9\n",
      "Epoch 1/30\n",
      "38/37 [==============================] - 7s 175ms/step - loss: 0.8282 - acc: 0.5932 - val_loss: 0.5762 - val_acc: 0.6591\n",
      "Epoch 2/30\n",
      "38/37 [==============================] - 4s 117ms/step - loss: 0.6190 - acc: 0.6923 - val_loss: 0.4137 - val_acc: 0.8333\n",
      "Epoch 3/30\n",
      "38/37 [==============================] - 4s 117ms/step - loss: 0.5094 - acc: 0.7519 - val_loss: 0.4660 - val_acc: 0.7500\n",
      "Epoch 4/30\n",
      "38/37 [==============================] - 4s 117ms/step - loss: 0.4552 - acc: 0.7668 - val_loss: 0.4521 - val_acc: 0.7500\n",
      "Epoch 5/30\n",
      "38/37 [==============================] - 4s 117ms/step - loss: 0.4248 - acc: 0.7993 - val_loss: 0.3582 - val_acc: 0.8485\n",
      "Epoch 6/30\n",
      "38/37 [==============================] - 4s 118ms/step - loss: 0.3737 - acc: 0.8169 - val_loss: 0.4158 - val_acc: 0.8030\n",
      "Epoch 7/30\n",
      "38/37 [==============================] - 4s 118ms/step - loss: 0.2969 - acc: 0.8647 - val_loss: 0.2640 - val_acc: 0.8939\n",
      "Epoch 8/30\n",
      "38/37 [==============================] - 4s 118ms/step - loss: 0.3260 - acc: 0.8499 - val_loss: 0.3459 - val_acc: 0.8409\n",
      "Epoch 9/30\n",
      "38/37 [==============================] - 4s 117ms/step - loss: 0.2988 - acc: 0.8692 - val_loss: 0.2832 - val_acc: 0.8864\n",
      "Epoch 10/30\n",
      "38/37 [==============================] - 4s 117ms/step - loss: 0.3235 - acc: 0.8540 - val_loss: 0.3237 - val_acc: 0.8712\n",
      "Epoch 11/30\n",
      "38/37 [==============================] - 4s 116ms/step - loss: 0.2786 - acc: 0.8836 - val_loss: 0.2361 - val_acc: 0.8788\n",
      "Epoch 12/30\n",
      "38/37 [==============================] - 4s 116ms/step - loss: 0.2416 - acc: 0.9017 - val_loss: 0.2828 - val_acc: 0.8788\n",
      "Epoch 13/30\n",
      "38/37 [==============================] - 4s 117ms/step - loss: 0.2734 - acc: 0.8922 - val_loss: 0.4519 - val_acc: 0.8258\n",
      "Epoch 14/30\n",
      "38/37 [==============================] - 4s 118ms/step - loss: 0.2941 - acc: 0.8642 - val_loss: 0.4782 - val_acc: 0.7652\n",
      "Epoch 15/30\n",
      "38/37 [==============================] - 4s 117ms/step - loss: 0.2260 - acc: 0.9136 - val_loss: 0.3087 - val_acc: 0.8636\n",
      "Epoch 16/30\n",
      "38/37 [==============================] - 4s 117ms/step - loss: 0.2231 - acc: 0.9050 - val_loss: 0.2305 - val_acc: 0.8864\n",
      "Epoch 17/30\n",
      "38/37 [==============================] - 4s 117ms/step - loss: 0.2123 - acc: 0.9116 - val_loss: 0.4265 - val_acc: 0.8333\n",
      "Epoch 18/30\n",
      "38/37 [==============================] - 4s 118ms/step - loss: 0.2330 - acc: 0.8976 - val_loss: 0.2558 - val_acc: 0.8864\n",
      "Epoch 19/30\n",
      "38/37 [==============================] - 4s 117ms/step - loss: 0.1979 - acc: 0.9042 - val_loss: 0.3201 - val_acc: 0.8636\n",
      "Epoch 20/30\n",
      "38/37 [==============================] - 4s 117ms/step - loss: 0.2332 - acc: 0.8993 - val_loss: 0.7938 - val_acc: 0.7500\n",
      "Epoch 21/30\n",
      "38/37 [==============================] - 4s 117ms/step - loss: 0.2774 - acc: 0.8790 - val_loss: 0.3273 - val_acc: 0.8485\n",
      "Epoch 22/30\n",
      "38/37 [==============================] - 4s 117ms/step - loss: 0.2160 - acc: 0.9087 - val_loss: 0.2605 - val_acc: 0.8788\n",
      "Epoch 23/30\n",
      "38/37 [==============================] - 4s 116ms/step - loss: 0.1930 - acc: 0.9243 - val_loss: 0.2186 - val_acc: 0.9091\n",
      "Epoch 24/30\n",
      "38/37 [==============================] - 4s 117ms/step - loss: 0.2190 - acc: 0.8993 - val_loss: 0.5460 - val_acc: 0.8182\n",
      "Epoch 25/30\n",
      "38/37 [==============================] - 4s 116ms/step - loss: 0.2290 - acc: 0.9021 - val_loss: 0.2514 - val_acc: 0.8939\n",
      "Epoch 26/30\n",
      "38/37 [==============================] - 4s 116ms/step - loss: 0.1825 - acc: 0.9243 - val_loss: 0.2940 - val_acc: 0.8485\n",
      "Epoch 27/30\n",
      "38/37 [==============================] - 4s 116ms/step - loss: 0.1652 - acc: 0.9297 - val_loss: 0.2434 - val_acc: 0.8939\n",
      "Epoch 28/30\n",
      "38/37 [==============================] - 4s 117ms/step - loss: 0.1591 - acc: 0.9334 - val_loss: 0.4790 - val_acc: 0.8636\n",
      "Epoch 29/30\n",
      "38/37 [==============================] - 4s 117ms/step - loss: 0.1710 - acc: 0.9276 - val_loss: 0.2568 - val_acc: 0.8788\n",
      "Epoch 30/30\n",
      "38/37 [==============================] - 4s 117ms/step - loss: 0.1753 - acc: 0.9243 - val_loss: 0.3079 - val_acc: 0.8561\n",
      "132/132 [==============================] - 0s 1ms/step\n",
      "\n",
      " Val score: 0.21857781279267688\n",
      "\n",
      " Val accuracy: 0.9090909090909091\n",
      "         id    is_iceberg\n",
      "0  d4bb8bac  9.976180e-01\n",
      "1  28612c22  5.073809e-07\n",
      "2  d5c8ab27  1.566053e-05\n",
      "3  b8cf5edb  5.741748e-01\n",
      "4  3f6778c5  4.962790e-01\n",
      "5  0ecf7d79  5.293955e-07\n",
      "6  4acfe35a  9.217428e-01\n",
      "7  a5b4d551  2.157824e-07\n",
      "8  cacbe718  9.991560e-01\n",
      "9  de1244d8  9.864161e-01\n",
      "submission saved\n"
     ]
    }
   ],
   "source": [
    "eva_score = []\n",
    "\n",
    "#train model using 10cv\n",
    "for j, (train_idx, val_idx) in enumerate(folds):\n",
    "    \n",
    "    print('\\nFold ',j)\n",
    "    X_train_cv = X_train[train_idx]\n",
    "    y_train_cv = y_train[train_idx]\n",
    "    X_valid_cv = X_train[val_idx]\n",
    "    y_valid_cv= y_train[val_idx]\n",
    "    \n",
    "    name_weights = \"final_model_fold\" + str(j) + \"_weights.h5\"\n",
    "    callbacks = get_callbacks(name_weights = name_weights, patience_lr=15)\n",
    "    generator = gen.flow(X_train_cv, y_train_cv, batch_size = batch_size)\n",
    "    model = get_model()\n",
    "    model.fit_generator(\n",
    "                generator,\n",
    "                steps_per_epoch=len(X_train_cv)/batch_size,\n",
    "                epochs=30,\n",
    "                verbose=1,\n",
    "                validation_data = (X_valid_cv, y_valid_cv),\n",
    "                callbacks = callbacks)\n",
    "    \n",
    "    model.load_weights(filepath = name_weights)\n",
    "    \n",
    "    score = model.evaluate(X_valid_cv, y_valid_cv, verbose=1)\n",
    "    print('\\n Val score:', score[0])\n",
    "    print('\\n Val accuracy:', score[1])\n",
    "    \n",
    "    eva_score.append(score[1])\n",
    "\n",
    "    SUBMISSION = './result/net1/sub_simple_v1_{}.csv'.format(j)\n",
    "\n",
    "    pred_test = model.predict(X_test)\n",
    "    id = np.array(train['id'])\n",
    "    id = id[-148:]\n",
    "    \n",
    "    submission = pd.DataFrame({'id': id ,'is_iceberg': pred_test.reshape((pred_test.shape[0]))})\n",
    "    print(submission.head(10))\n",
    "\n",
    "    submission.to_csv(SUBMISSION, index=False)\n",
    "    print(\"submission saved\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Ensemble Using Stacking For Fine-tuning and Experiment"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 11,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/html": [
       "<div>\n",
       "<style scoped>\n",
       "    .dataframe tbody tr th:only-of-type {\n",
       "        vertical-align: middle;\n",
       "    }\n",
       "\n",
       "    .dataframe tbody tr th {\n",
       "        vertical-align: top;\n",
       "    }\n",
       "\n",
       "    .dataframe thead th {\n",
       "        text-align: right;\n",
       "    }\n",
       "</style>\n",
       "<table border=\"1\" class=\"dataframe\">\n",
       "  <thead>\n",
       "    <tr style=\"text-align: right;\">\n",
       "      <th></th>\n",
       "      <th>id</th>\n",
       "      <th>is_iceberg_0</th>\n",
       "      <th>is_iceberg_1</th>\n",
       "      <th>is_iceberg_2</th>\n",
       "      <th>is_iceberg_3</th>\n",
       "      <th>is_iceberg_4</th>\n",
       "      <th>is_iceberg_5</th>\n",
       "      <th>is_iceberg_6</th>\n",
       "      <th>is_iceberg_7</th>\n",
       "      <th>is_iceberg_8</th>\n",
       "      <th>is_iceberg_9</th>\n",
       "    </tr>\n",
       "  </thead>\n",
       "  <tbody>\n",
       "    <tr>\n",
       "      <th>0</th>\n",
       "      <td>d4bb8bac</td>\n",
       "      <td>0.973668</td>\n",
       "      <td>9.976180e-01</td>\n",
       "      <td>0.964356</td>\n",
       "      <td>0.885859</td>\n",
       "      <td>0.930919</td>\n",
       "      <td>9.194921e-01</td>\n",
       "      <td>0.947607</td>\n",
       "      <td>0.988759</td>\n",
       "      <td>0.989776</td>\n",
       "      <td>9.441719e-01</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>1</th>\n",
       "      <td>28612c22</td>\n",
       "      <td>0.000006</td>\n",
       "      <td>5.073809e-07</td>\n",
       "      <td>0.000031</td>\n",
       "      <td>0.000006</td>\n",
       "      <td>0.000057</td>\n",
       "      <td>6.721845e-08</td>\n",
       "      <td>0.000104</td>\n",
       "      <td>0.000002</td>\n",
       "      <td>0.000004</td>\n",
       "      <td>6.585086e-08</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>2</th>\n",
       "      <td>d5c8ab27</td>\n",
       "      <td>0.001980</td>\n",
       "      <td>1.566053e-05</td>\n",
       "      <td>0.000477</td>\n",
       "      <td>0.000650</td>\n",
       "      <td>0.000110</td>\n",
       "      <td>8.144771e-06</td>\n",
       "      <td>0.000031</td>\n",
       "      <td>0.000678</td>\n",
       "      <td>0.019846</td>\n",
       "      <td>1.033631e-04</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>3</th>\n",
       "      <td>b8cf5edb</td>\n",
       "      <td>0.288146</td>\n",
       "      <td>5.741748e-01</td>\n",
       "      <td>0.230417</td>\n",
       "      <td>0.510641</td>\n",
       "      <td>0.302515</td>\n",
       "      <td>1.431399e-01</td>\n",
       "      <td>0.664554</td>\n",
       "      <td>0.515260</td>\n",
       "      <td>0.491432</td>\n",
       "      <td>6.893286e-03</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>4</th>\n",
       "      <td>3f6778c5</td>\n",
       "      <td>0.360043</td>\n",
       "      <td>4.962790e-01</td>\n",
       "      <td>0.372624</td>\n",
       "      <td>0.600870</td>\n",
       "      <td>0.534991</td>\n",
       "      <td>1.799062e-01</td>\n",
       "      <td>0.453289</td>\n",
       "      <td>0.617451</td>\n",
       "      <td>0.356469</td>\n",
       "      <td>1.513270e-01</td>\n",
       "    </tr>\n",
       "  </tbody>\n",
       "</table>\n",
       "</div>"
      ],
      "text/plain": [
       "         id  is_iceberg_0  is_iceberg_1  is_iceberg_2  is_iceberg_3  \\\n",
       "0  d4bb8bac      0.973668  9.976180e-01      0.964356      0.885859   \n",
       "1  28612c22      0.000006  5.073809e-07      0.000031      0.000006   \n",
       "2  d5c8ab27      0.001980  1.566053e-05      0.000477      0.000650   \n",
       "3  b8cf5edb      0.288146  5.741748e-01      0.230417      0.510641   \n",
       "4  3f6778c5      0.360043  4.962790e-01      0.372624      0.600870   \n",
       "\n",
       "   is_iceberg_4  is_iceberg_5  is_iceberg_6  is_iceberg_7  is_iceberg_8  \\\n",
       "0      0.930919  9.194921e-01      0.947607      0.988759      0.989776   \n",
       "1      0.000057  6.721845e-08      0.000104      0.000002      0.000004   \n",
       "2      0.000110  8.144771e-06      0.000031      0.000678      0.019846   \n",
       "3      0.302515  1.431399e-01      0.664554      0.515260      0.491432   \n",
       "4      0.534991  1.799062e-01      0.453289      0.617451      0.356469   \n",
       "\n",
       "   is_iceberg_9  \n",
       "0  9.441719e-01  \n",
       "1  6.585086e-08  \n",
       "2  1.033631e-04  \n",
       "3  6.893286e-03  \n",
       "4  1.513270e-01  "
      ]
     },
     "execution_count": 11,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "import os\n",
    "sub_path = \"./result/net1/\"\n",
    "all_files = os.listdir(sub_path)\n",
    "\n",
    "# Read and concatenate submissions\n",
    "outs = [pd.read_csv(os.path.join(sub_path, f), index_col=0) for f in all_files]\n",
    "concat_sub = pd.concat(outs, axis=1)\n",
    "cols = list(map(lambda x: \"is_iceberg_\" + str(x), range(len(concat_sub.columns))))\n",
    "concat_sub.columns = cols\n",
    "concat_sub.reset_index(inplace=True)\n",
    "concat_sub.head()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 12,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/html": [
       "<div>\n",
       "<style scoped>\n",
       "    .dataframe tbody tr th:only-of-type {\n",
       "        vertical-align: middle;\n",
       "    }\n",
       "\n",
       "    .dataframe tbody tr th {\n",
       "        vertical-align: top;\n",
       "    }\n",
       "\n",
       "    .dataframe thead th {\n",
       "        text-align: right;\n",
       "    }\n",
       "</style>\n",
       "<table border=\"1\" class=\"dataframe\">\n",
       "  <thead>\n",
       "    <tr style=\"text-align: right;\">\n",
       "      <th></th>\n",
       "      <th>is_iceberg_0</th>\n",
       "      <th>is_iceberg_1</th>\n",
       "      <th>is_iceberg_2</th>\n",
       "      <th>is_iceberg_3</th>\n",
       "      <th>is_iceberg_4</th>\n",
       "      <th>is_iceberg_5</th>\n",
       "      <th>is_iceberg_6</th>\n",
       "      <th>is_iceberg_7</th>\n",
       "      <th>is_iceberg_8</th>\n",
       "      <th>is_iceberg_9</th>\n",
       "    </tr>\n",
       "  </thead>\n",
       "  <tbody>\n",
       "    <tr>\n",
       "      <th>is_iceberg_0</th>\n",
       "      <td>1.000000</td>\n",
       "      <td>0.969863</td>\n",
       "      <td>0.955275</td>\n",
       "      <td>0.964320</td>\n",
       "      <td>0.982152</td>\n",
       "      <td>0.962479</td>\n",
       "      <td>0.965361</td>\n",
       "      <td>0.968899</td>\n",
       "      <td>0.946903</td>\n",
       "      <td>0.941480</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>is_iceberg_1</th>\n",
       "      <td>0.969863</td>\n",
       "      <td>1.000000</td>\n",
       "      <td>0.965910</td>\n",
       "      <td>0.977433</td>\n",
       "      <td>0.968359</td>\n",
       "      <td>0.950655</td>\n",
       "      <td>0.976124</td>\n",
       "      <td>0.980424</td>\n",
       "      <td>0.963287</td>\n",
       "      <td>0.910391</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>is_iceberg_2</th>\n",
       "      <td>0.955275</td>\n",
       "      <td>0.965910</td>\n",
       "      <td>1.000000</td>\n",
       "      <td>0.946215</td>\n",
       "      <td>0.961264</td>\n",
       "      <td>0.966244</td>\n",
       "      <td>0.969385</td>\n",
       "      <td>0.967788</td>\n",
       "      <td>0.962165</td>\n",
       "      <td>0.943969</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>is_iceberg_3</th>\n",
       "      <td>0.964320</td>\n",
       "      <td>0.977433</td>\n",
       "      <td>0.946215</td>\n",
       "      <td>1.000000</td>\n",
       "      <td>0.962491</td>\n",
       "      <td>0.939466</td>\n",
       "      <td>0.962754</td>\n",
       "      <td>0.974122</td>\n",
       "      <td>0.965960</td>\n",
       "      <td>0.898984</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>is_iceberg_4</th>\n",
       "      <td>0.982152</td>\n",
       "      <td>0.968359</td>\n",
       "      <td>0.961264</td>\n",
       "      <td>0.962491</td>\n",
       "      <td>1.000000</td>\n",
       "      <td>0.955247</td>\n",
       "      <td>0.975366</td>\n",
       "      <td>0.970784</td>\n",
       "      <td>0.961115</td>\n",
       "      <td>0.935896</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>is_iceberg_5</th>\n",
       "      <td>0.962479</td>\n",
       "      <td>0.950655</td>\n",
       "      <td>0.966244</td>\n",
       "      <td>0.939466</td>\n",
       "      <td>0.955247</td>\n",
       "      <td>1.000000</td>\n",
       "      <td>0.972146</td>\n",
       "      <td>0.944780</td>\n",
       "      <td>0.933344</td>\n",
       "      <td>0.968627</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>is_iceberg_6</th>\n",
       "      <td>0.965361</td>\n",
       "      <td>0.976124</td>\n",
       "      <td>0.969385</td>\n",
       "      <td>0.962754</td>\n",
       "      <td>0.975366</td>\n",
       "      <td>0.972146</td>\n",
       "      <td>1.000000</td>\n",
       "      <td>0.969456</td>\n",
       "      <td>0.957720</td>\n",
       "      <td>0.948308</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>is_iceberg_7</th>\n",
       "      <td>0.968899</td>\n",
       "      <td>0.980424</td>\n",
       "      <td>0.967788</td>\n",
       "      <td>0.974122</td>\n",
       "      <td>0.970784</td>\n",
       "      <td>0.944780</td>\n",
       "      <td>0.969456</td>\n",
       "      <td>1.000000</td>\n",
       "      <td>0.966638</td>\n",
       "      <td>0.919727</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>is_iceberg_8</th>\n",
       "      <td>0.946903</td>\n",
       "      <td>0.963287</td>\n",
       "      <td>0.962165</td>\n",
       "      <td>0.965960</td>\n",
       "      <td>0.961115</td>\n",
       "      <td>0.933344</td>\n",
       "      <td>0.957720</td>\n",
       "      <td>0.966638</td>\n",
       "      <td>1.000000</td>\n",
       "      <td>0.898580</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>is_iceberg_9</th>\n",
       "      <td>0.941480</td>\n",
       "      <td>0.910391</td>\n",
       "      <td>0.943969</td>\n",
       "      <td>0.898984</td>\n",
       "      <td>0.935896</td>\n",
       "      <td>0.968627</td>\n",
       "      <td>0.948308</td>\n",
       "      <td>0.919727</td>\n",
       "      <td>0.898580</td>\n",
       "      <td>1.000000</td>\n",
       "    </tr>\n",
       "  </tbody>\n",
       "</table>\n",
       "</div>"
      ],
      "text/plain": [
       "              is_iceberg_0  is_iceberg_1  is_iceberg_2  is_iceberg_3  \\\n",
       "is_iceberg_0      1.000000      0.969863      0.955275      0.964320   \n",
       "is_iceberg_1      0.969863      1.000000      0.965910      0.977433   \n",
       "is_iceberg_2      0.955275      0.965910      1.000000      0.946215   \n",
       "is_iceberg_3      0.964320      0.977433      0.946215      1.000000   \n",
       "is_iceberg_4      0.982152      0.968359      0.961264      0.962491   \n",
       "is_iceberg_5      0.962479      0.950655      0.966244      0.939466   \n",
       "is_iceberg_6      0.965361      0.976124      0.969385      0.962754   \n",
       "is_iceberg_7      0.968899      0.980424      0.967788      0.974122   \n",
       "is_iceberg_8      0.946903      0.963287      0.962165      0.965960   \n",
       "is_iceberg_9      0.941480      0.910391      0.943969      0.898984   \n",
       "\n",
       "              is_iceberg_4  is_iceberg_5  is_iceberg_6  is_iceberg_7  \\\n",
       "is_iceberg_0      0.982152      0.962479      0.965361      0.968899   \n",
       "is_iceberg_1      0.968359      0.950655      0.976124      0.980424   \n",
       "is_iceberg_2      0.961264      0.966244      0.969385      0.967788   \n",
       "is_iceberg_3      0.962491      0.939466      0.962754      0.974122   \n",
       "is_iceberg_4      1.000000      0.955247      0.975366      0.970784   \n",
       "is_iceberg_5      0.955247      1.000000      0.972146      0.944780   \n",
       "is_iceberg_6      0.975366      0.972146      1.000000      0.969456   \n",
       "is_iceberg_7      0.970784      0.944780      0.969456      1.000000   \n",
       "is_iceberg_8      0.961115      0.933344      0.957720      0.966638   \n",
       "is_iceberg_9      0.935896      0.968627      0.948308      0.919727   \n",
       "\n",
       "              is_iceberg_8  is_iceberg_9  \n",
       "is_iceberg_0      0.946903      0.941480  \n",
       "is_iceberg_1      0.963287      0.910391  \n",
       "is_iceberg_2      0.962165      0.943969  \n",
       "is_iceberg_3      0.965960      0.898984  \n",
       "is_iceberg_4      0.961115      0.935896  \n",
       "is_iceberg_5      0.933344      0.968627  \n",
       "is_iceberg_6      0.957720      0.948308  \n",
       "is_iceberg_7      0.966638      0.919727  \n",
       "is_iceberg_8      1.000000      0.898580  \n",
       "is_iceberg_9      0.898580      1.000000  "
      ]
     },
     "execution_count": 12,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "concat_sub.corr()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 13,
   "metadata": {},
   "outputs": [],
   "source": [
    "# get the data fields ready for stacking\n",
    "concat_sub['is_iceberg_max'] = concat_sub.iloc[:, 1:11].max(axis=1)\n",
    "concat_sub['is_iceberg_min'] = concat_sub.iloc[:, 1:11].min(axis=1)\n",
    "concat_sub['is_iceberg_mean'] = concat_sub.iloc[:, 1:11].mean(axis=1)\n",
    "concat_sub['is_iceberg_median'] = concat_sub.iloc[:, 1:11].median(axis=1)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 14,
   "metadata": {},
   "outputs": [],
   "source": [
    "# set up cutoff threshold for lower and upper bounds, easy to twist \n",
    "cutoff_lo = 0.8\n",
    "cutoff_hi = 0.2"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 15,
   "metadata": {},
   "outputs": [],
   "source": [
    "concat_sub['is_iceberg'] = concat_sub['is_iceberg_median']\n",
    "concat_sub[['id', 'is_iceberg']].to_csv('./result/ensemble/stack_median82.csv', \n",
    "                                        index=False, float_format='%.6f')"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 16,
   "metadata": {},
   "outputs": [],
   "source": [
    "concat_sub['is_iceberg'] = np.where(np.all(concat_sub.iloc[:,1:11] > cutoff_lo, axis=1), \n",
    "                                    concat_sub['is_iceberg_max'], \n",
    "                                    np.where(np.all(concat_sub.iloc[:,1:11] < cutoff_hi, axis=1),\n",
    "                                             concat_sub['is_iceberg_min'], \n",
    "                                             concat_sub['is_iceberg_median']))\n",
    "concat_sub[['id', 'is_iceberg']].to_csv('./result/ensemble/stack_minmax_median82.csv', \n",
    "                                        index=False, float_format='%.6f')\n"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 17,
   "metadata": {},
   "outputs": [],
   "source": [
    "concat_sub['is_iceberg'] = np.where(np.all(concat_sub.iloc[:,1:11] > cutoff_lo, axis=1), \n",
    "                                    concat_sub['is_iceberg_max'], \n",
    "                                    np.where(np.all(concat_sub.iloc[:,1:11] < cutoff_hi, axis=1),\n",
    "                                             concat_sub['is_iceberg_min'], \n",
    "                                             concat_sub['is_iceberg_mean']))\n",
    "concat_sub[['id', 'is_iceberg']].to_csv('./result/ensemble/stack_minmax_mean82.csv', \n",
    "                                        index=False, float_format='%.6f')"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 18,
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "0.18124065184350735\n",
      "0.17356563428940697\n",
      "0.17288809474612515\n"
     ]
    }
   ],
   "source": [
    "stack_median = pd.read_csv('./result/ensemble/stack_median82.csv')\n",
    "stack_minmax = pd.read_csv('./result/ensemble/stack_minmax_median82.csv')\n",
    "stack_minmax2 = pd.read_csv('./result/ensemble/stack_minmax_mean82.csv')\n",
    "\n",
    "from sklearn.metrics import log_loss\n",
    "print(log_loss(y_test, stack_median['is_iceberg']))\n",
    "print(log_loss(y_test, stack_minmax['is_iceberg']))\n",
    "print(log_loss(y_test, stack_minmax2['is_iceberg']))\n",
    "\n",
    "stack_median = (stack_median > 0.5)\n",
    "stack_minmax = (stack_minmax > 0.5)\n",
    "stack_minmax2 = (stack_minmax2 > 0.5)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 32,
   "metadata": {
    "scrolled": true
   },
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "0.9121621621621622\n",
      "0.9121621621621622\n",
      "0.9054054054054054\n"
     ]
    }
   ],
   "source": [
    "from sklearn.metrics import accuracy_score\n",
    "print(accuracy_score(stack_median['is_iceberg'], y_test))\n",
    "print(accuracy_score(stack_minmax['is_iceberg'], y_test))\n",
    "print(accuracy_score(stack_minmax2['is_iceberg'], y_test))"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Majority Voting"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 94,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/html": [
       "<div>\n",
       "<style scoped>\n",
       "    .dataframe tbody tr th:only-of-type {\n",
       "        vertical-align: middle;\n",
       "    }\n",
       "\n",
       "    .dataframe tbody tr th {\n",
       "        vertical-align: top;\n",
       "    }\n",
       "\n",
       "    .dataframe thead th {\n",
       "        text-align: right;\n",
       "    }\n",
       "</style>\n",
       "<table border=\"1\" class=\"dataframe\">\n",
       "  <thead>\n",
       "    <tr style=\"text-align: right;\">\n",
       "      <th></th>\n",
       "      <th>id</th>\n",
       "      <th>is_iceberg_0</th>\n",
       "      <th>is_iceberg_1</th>\n",
       "      <th>is_iceberg_2</th>\n",
       "      <th>is_iceberg_3</th>\n",
       "      <th>is_iceberg_4</th>\n",
       "      <th>is_iceberg_5</th>\n",
       "      <th>is_iceberg_6</th>\n",
       "      <th>is_iceberg_7</th>\n",
       "      <th>is_iceberg_8</th>\n",
       "      <th>is_iceberg_9</th>\n",
       "    </tr>\n",
       "  </thead>\n",
       "  <tbody>\n",
       "    <tr>\n",
       "      <th>0</th>\n",
       "      <td>d4bb8bac</td>\n",
       "      <td>0.998125</td>\n",
       "      <td>0.995539</td>\n",
       "      <td>0.999661</td>\n",
       "      <td>0.991629</td>\n",
       "      <td>0.975994</td>\n",
       "      <td>0.586749</td>\n",
       "      <td>0.996489</td>\n",
       "      <td>0.871387</td>\n",
       "      <td>0.999085</td>\n",
       "      <td>9.927993e-01</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>1</th>\n",
       "      <td>28612c22</td>\n",
       "      <td>0.000045</td>\n",
       "      <td>0.000002</td>\n",
       "      <td>0.000023</td>\n",
       "      <td>0.000043</td>\n",
       "      <td>0.000003</td>\n",
       "      <td>0.000027</td>\n",
       "      <td>0.000119</td>\n",
       "      <td>0.000077</td>\n",
       "      <td>0.000016</td>\n",
       "      <td>5.271390e-08</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>2</th>\n",
       "      <td>d5c8ab27</td>\n",
       "      <td>0.000974</td>\n",
       "      <td>0.000127</td>\n",
       "      <td>0.006496</td>\n",
       "      <td>0.000271</td>\n",
       "      <td>0.000005</td>\n",
       "      <td>0.000147</td>\n",
       "      <td>0.024394</td>\n",
       "      <td>0.008605</td>\n",
       "      <td>0.000344</td>\n",
       "      <td>2.275544e-04</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>3</th>\n",
       "      <td>b8cf5edb</td>\n",
       "      <td>0.383438</td>\n",
       "      <td>0.384115</td>\n",
       "      <td>0.490866</td>\n",
       "      <td>0.306225</td>\n",
       "      <td>0.106516</td>\n",
       "      <td>0.086882</td>\n",
       "      <td>0.438888</td>\n",
       "      <td>0.177845</td>\n",
       "      <td>0.285328</td>\n",
       "      <td>1.178622e-01</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>4</th>\n",
       "      <td>3f6778c5</td>\n",
       "      <td>0.385801</td>\n",
       "      <td>0.520272</td>\n",
       "      <td>0.152858</td>\n",
       "      <td>0.302401</td>\n",
       "      <td>0.259161</td>\n",
       "      <td>0.425656</td>\n",
       "      <td>0.466482</td>\n",
       "      <td>0.248833</td>\n",
       "      <td>0.265736</td>\n",
       "      <td>1.007535e-01</td>\n",
       "    </tr>\n",
       "  </tbody>\n",
       "</table>\n",
       "</div>"
      ],
      "text/plain": [
       "         id  is_iceberg_0  is_iceberg_1  is_iceberg_2  is_iceberg_3  \\\n",
       "0  d4bb8bac      0.998125      0.995539      0.999661      0.991629   \n",
       "1  28612c22      0.000045      0.000002      0.000023      0.000043   \n",
       "2  d5c8ab27      0.000974      0.000127      0.006496      0.000271   \n",
       "3  b8cf5edb      0.383438      0.384115      0.490866      0.306225   \n",
       "4  3f6778c5      0.385801      0.520272      0.152858      0.302401   \n",
       "\n",
       "   is_iceberg_4  is_iceberg_5  is_iceberg_6  is_iceberg_7  is_iceberg_8  \\\n",
       "0      0.975994      0.586749      0.996489      0.871387      0.999085   \n",
       "1      0.000003      0.000027      0.000119      0.000077      0.000016   \n",
       "2      0.000005      0.000147      0.024394      0.008605      0.000344   \n",
       "3      0.106516      0.086882      0.438888      0.177845      0.285328   \n",
       "4      0.259161      0.425656      0.466482      0.248833      0.265736   \n",
       "\n",
       "   is_iceberg_9  \n",
       "0  9.927993e-01  \n",
       "1  5.271390e-08  \n",
       "2  2.275544e-04  \n",
       "3  1.178622e-01  \n",
       "4  1.007535e-01  "
      ]
     },
     "execution_count": 94,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "import os\n",
    "sub_path = \"./result/net1/\"\n",
    "all_files = os.listdir(sub_path)\n",
    "\n",
    "# Read and concatenate submissions\n",
    "outs = [pd.read_csv(os.path.join(sub_path, f), index_col=0) for f in all_files]\n",
    "concat_sub = pd.concat(outs, axis=1)\n",
    "cols = list(map(lambda x: \"is_iceberg_\" + str(x), range(len(concat_sub.columns))))\n",
    "concat_sub.columns = cols\n",
    "concat_sub.reset_index(inplace=True)\n",
    "concat_sub.head()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 95,
   "metadata": {},
   "outputs": [],
   "source": [
    "matrix = np.array(concat_sub.iloc[:,1:9])\n",
    "ice_berg = []\n",
    "for i in matrix:\n",
    "    a = i\n",
    "    r1 = a[a >= 0.8]\n",
    "    r2 = a[a <= 0.2]\n",
    "    \n",
    "    result = np.concatenate((r1,r2), axis = 0)\n",
    "    result = result >= 0.5\n",
    "    \n",
    "    if np.sum(result) > len(result)/2.0:\n",
    "        ice_berg.append(1)\n",
    "    else:\n",
    "        ice_berg.append(0)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 96,
   "metadata": {},
   "outputs": [],
   "source": [
    "concat_sub['is_iceberg'] = ice_berg"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 97,
   "metadata": {},
   "outputs": [],
   "source": [
    "concat_sub[['id', 'is_iceberg']].to_csv('./result/ensemble/Majority_voting_91.csv', \n",
    "                                        index=False, float_format='%.6f')"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 98,
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "0.8918918918918919\n"
     ]
    }
   ],
   "source": [
    "majority = pd.read_csv('./result/ensemble/Majority_voting_91.csv')\n",
    "print(accuracy_score(majority['is_iceberg'], y_test))"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Combined Model"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 1,
   "metadata": {},
   "outputs": [],
   "source": [
    "import os\n",
    "import numpy as np \n",
    "import pandas as pd"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 2,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/html": [
       "<div>\n",
       "<style scoped>\n",
       "    .dataframe tbody tr th:only-of-type {\n",
       "        vertical-align: middle;\n",
       "    }\n",
       "\n",
       "    .dataframe tbody tr th {\n",
       "        vertical-align: top;\n",
       "    }\n",
       "\n",
       "    .dataframe thead th {\n",
       "        text-align: right;\n",
       "    }\n",
       "</style>\n",
       "<table border=\"1\" class=\"dataframe\">\n",
       "  <thead>\n",
       "    <tr style=\"text-align: right;\">\n",
       "      <th></th>\n",
       "      <th>id</th>\n",
       "      <th>is_iceberg_0</th>\n",
       "      <th>is_iceberg_1</th>\n",
       "      <th>is_iceberg_2</th>\n",
       "      <th>is_iceberg_3</th>\n",
       "      <th>is_iceberg_4</th>\n",
       "      <th>is_iceberg_5</th>\n",
       "      <th>is_iceberg_6</th>\n",
       "      <th>is_iceberg_7</th>\n",
       "      <th>is_iceberg_8</th>\n",
       "    </tr>\n",
       "  </thead>\n",
       "  <tbody>\n",
       "    <tr>\n",
       "      <th>0</th>\n",
       "      <td>d4bb8bac</td>\n",
       "      <td>0.999960</td>\n",
       "      <td>0.995394</td>\n",
       "      <td>0.990017</td>\n",
       "      <td>0.997618</td>\n",
       "      <td>0.999353</td>\n",
       "      <td>0.968913</td>\n",
       "      <td>0.994141</td>\n",
       "      <td>0.989210</td>\n",
       "      <td>0.997138</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>1</th>\n",
       "      <td>28612c22</td>\n",
       "      <td>0.000001</td>\n",
       "      <td>0.000001</td>\n",
       "      <td>0.000000</td>\n",
       "      <td>0.000000</td>\n",
       "      <td>0.000000</td>\n",
       "      <td>0.000000</td>\n",
       "      <td>0.000000</td>\n",
       "      <td>0.000029</td>\n",
       "      <td>0.000004</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>2</th>\n",
       "      <td>d5c8ab27</td>\n",
       "      <td>0.000004</td>\n",
       "      <td>0.000499</td>\n",
       "      <td>0.000123</td>\n",
       "      <td>0.000008</td>\n",
       "      <td>0.000001</td>\n",
       "      <td>0.000310</td>\n",
       "      <td>0.000028</td>\n",
       "      <td>0.001363</td>\n",
       "      <td>0.000748</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>3</th>\n",
       "      <td>b8cf5edb</td>\n",
       "      <td>0.346536</td>\n",
       "      <td>0.229875</td>\n",
       "      <td>0.198282</td>\n",
       "      <td>0.396974</td>\n",
       "      <td>0.350931</td>\n",
       "      <td>0.167257</td>\n",
       "      <td>0.329225</td>\n",
       "      <td>0.213739</td>\n",
       "      <td>0.311895</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>4</th>\n",
       "      <td>3f6778c5</td>\n",
       "      <td>0.424188</td>\n",
       "      <td>0.515291</td>\n",
       "      <td>0.345524</td>\n",
       "      <td>0.412957</td>\n",
       "      <td>0.302435</td>\n",
       "      <td>0.298849</td>\n",
       "      <td>0.436417</td>\n",
       "      <td>0.290727</td>\n",
       "      <td>0.419077</td>\n",
       "    </tr>\n",
       "  </tbody>\n",
       "</table>\n",
       "</div>"
      ],
      "text/plain": [
       "         id  is_iceberg_0  is_iceberg_1  is_iceberg_2  is_iceberg_3  \\\n",
       "0  d4bb8bac      0.999960      0.995394      0.990017      0.997618   \n",
       "1  28612c22      0.000001      0.000001      0.000000      0.000000   \n",
       "2  d5c8ab27      0.000004      0.000499      0.000123      0.000008   \n",
       "3  b8cf5edb      0.346536      0.229875      0.198282      0.396974   \n",
       "4  3f6778c5      0.424188      0.515291      0.345524      0.412957   \n",
       "\n",
       "   is_iceberg_4  is_iceberg_5  is_iceberg_6  is_iceberg_7  is_iceberg_8  \n",
       "0      0.999353      0.968913      0.994141      0.989210      0.997138  \n",
       "1      0.000000      0.000000      0.000000      0.000029      0.000004  \n",
       "2      0.000001      0.000310      0.000028      0.001363      0.000748  \n",
       "3      0.350931      0.167257      0.329225      0.213739      0.311895  \n",
       "4      0.302435      0.298849      0.436417      0.290727      0.419077  "
      ]
     },
     "execution_count": 2,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "sub_path = \"./result/ensemble4/\"\n",
    "all_files = os.listdir(sub_path)\n",
    "\n",
    "# Read and concatenate submissions\n",
    "outs = [pd.read_csv(os.path.join(sub_path, f), index_col=0) for f in all_files]\n",
    "concat_sub = pd.concat(outs, axis=1)\n",
    "cols = list(map(lambda x: \"is_iceberg_\" + str(x), range(len(concat_sub.columns))))\n",
    "concat_sub.columns = cols\n",
    "concat_sub.reset_index(inplace=True)\n",
    "concat_sub.head()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 3,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/html": [
       "<div>\n",
       "<style scoped>\n",
       "    .dataframe tbody tr th:only-of-type {\n",
       "        vertical-align: middle;\n",
       "    }\n",
       "\n",
       "    .dataframe tbody tr th {\n",
       "        vertical-align: top;\n",
       "    }\n",
       "\n",
       "    .dataframe thead th {\n",
       "        text-align: right;\n",
       "    }\n",
       "</style>\n",
       "<table border=\"1\" class=\"dataframe\">\n",
       "  <thead>\n",
       "    <tr style=\"text-align: right;\">\n",
       "      <th></th>\n",
       "      <th>is_iceberg_0</th>\n",
       "      <th>is_iceberg_1</th>\n",
       "      <th>is_iceberg_2</th>\n",
       "      <th>is_iceberg_3</th>\n",
       "      <th>is_iceberg_4</th>\n",
       "      <th>is_iceberg_5</th>\n",
       "      <th>is_iceberg_6</th>\n",
       "      <th>is_iceberg_7</th>\n",
       "      <th>is_iceberg_8</th>\n",
       "    </tr>\n",
       "  </thead>\n",
       "  <tbody>\n",
       "    <tr>\n",
       "      <th>is_iceberg_0</th>\n",
       "      <td>1.000000</td>\n",
       "      <td>0.984851</td>\n",
       "      <td>0.980860</td>\n",
       "      <td>0.995426</td>\n",
       "      <td>0.984782</td>\n",
       "      <td>0.986956</td>\n",
       "      <td>0.995450</td>\n",
       "      <td>0.984521</td>\n",
       "      <td>0.987171</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>is_iceberg_1</th>\n",
       "      <td>0.984851</td>\n",
       "      <td>1.000000</td>\n",
       "      <td>0.995399</td>\n",
       "      <td>0.985134</td>\n",
       "      <td>0.979440</td>\n",
       "      <td>0.992183</td>\n",
       "      <td>0.991377</td>\n",
       "      <td>0.989990</td>\n",
       "      <td>0.996423</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>is_iceberg_2</th>\n",
       "      <td>0.980860</td>\n",
       "      <td>0.995399</td>\n",
       "      <td>1.000000</td>\n",
       "      <td>0.983386</td>\n",
       "      <td>0.981107</td>\n",
       "      <td>0.992097</td>\n",
       "      <td>0.986953</td>\n",
       "      <td>0.989658</td>\n",
       "      <td>0.995483</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>is_iceberg_3</th>\n",
       "      <td>0.995426</td>\n",
       "      <td>0.985134</td>\n",
       "      <td>0.983386</td>\n",
       "      <td>1.000000</td>\n",
       "      <td>0.991824</td>\n",
       "      <td>0.990240</td>\n",
       "      <td>0.995181</td>\n",
       "      <td>0.985995</td>\n",
       "      <td>0.986247</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>is_iceberg_4</th>\n",
       "      <td>0.984782</td>\n",
       "      <td>0.979440</td>\n",
       "      <td>0.981107</td>\n",
       "      <td>0.991824</td>\n",
       "      <td>1.000000</td>\n",
       "      <td>0.990265</td>\n",
       "      <td>0.987358</td>\n",
       "      <td>0.982801</td>\n",
       "      <td>0.979320</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>is_iceberg_5</th>\n",
       "      <td>0.986956</td>\n",
       "      <td>0.992183</td>\n",
       "      <td>0.992097</td>\n",
       "      <td>0.990240</td>\n",
       "      <td>0.990265</td>\n",
       "      <td>1.000000</td>\n",
       "      <td>0.990798</td>\n",
       "      <td>0.989956</td>\n",
       "      <td>0.990099</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>is_iceberg_6</th>\n",
       "      <td>0.995450</td>\n",
       "      <td>0.991377</td>\n",
       "      <td>0.986953</td>\n",
       "      <td>0.995181</td>\n",
       "      <td>0.987358</td>\n",
       "      <td>0.990798</td>\n",
       "      <td>1.000000</td>\n",
       "      <td>0.988092</td>\n",
       "      <td>0.991532</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>is_iceberg_7</th>\n",
       "      <td>0.984521</td>\n",
       "      <td>0.989990</td>\n",
       "      <td>0.989658</td>\n",
       "      <td>0.985995</td>\n",
       "      <td>0.982801</td>\n",
       "      <td>0.989956</td>\n",
       "      <td>0.988092</td>\n",
       "      <td>1.000000</td>\n",
       "      <td>0.988391</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>is_iceberg_8</th>\n",
       "      <td>0.987171</td>\n",
       "      <td>0.996423</td>\n",
       "      <td>0.995483</td>\n",
       "      <td>0.986247</td>\n",
       "      <td>0.979320</td>\n",
       "      <td>0.990099</td>\n",
       "      <td>0.991532</td>\n",
       "      <td>0.988391</td>\n",
       "      <td>1.000000</td>\n",
       "    </tr>\n",
       "  </tbody>\n",
       "</table>\n",
       "</div>"
      ],
      "text/plain": [
       "              is_iceberg_0  is_iceberg_1  is_iceberg_2  is_iceberg_3  \\\n",
       "is_iceberg_0      1.000000      0.984851      0.980860      0.995426   \n",
       "is_iceberg_1      0.984851      1.000000      0.995399      0.985134   \n",
       "is_iceberg_2      0.980860      0.995399      1.000000      0.983386   \n",
       "is_iceberg_3      0.995426      0.985134      0.983386      1.000000   \n",
       "is_iceberg_4      0.984782      0.979440      0.981107      0.991824   \n",
       "is_iceberg_5      0.986956      0.992183      0.992097      0.990240   \n",
       "is_iceberg_6      0.995450      0.991377      0.986953      0.995181   \n",
       "is_iceberg_7      0.984521      0.989990      0.989658      0.985995   \n",
       "is_iceberg_8      0.987171      0.996423      0.995483      0.986247   \n",
       "\n",
       "              is_iceberg_4  is_iceberg_5  is_iceberg_6  is_iceberg_7  \\\n",
       "is_iceberg_0      0.984782      0.986956      0.995450      0.984521   \n",
       "is_iceberg_1      0.979440      0.992183      0.991377      0.989990   \n",
       "is_iceberg_2      0.981107      0.992097      0.986953      0.989658   \n",
       "is_iceberg_3      0.991824      0.990240      0.995181      0.985995   \n",
       "is_iceberg_4      1.000000      0.990265      0.987358      0.982801   \n",
       "is_iceberg_5      0.990265      1.000000      0.990798      0.989956   \n",
       "is_iceberg_6      0.987358      0.990798      1.000000      0.988092   \n",
       "is_iceberg_7      0.982801      0.989956      0.988092      1.000000   \n",
       "is_iceberg_8      0.979320      0.990099      0.991532      0.988391   \n",
       "\n",
       "              is_iceberg_8  \n",
       "is_iceberg_0      0.987171  \n",
       "is_iceberg_1      0.996423  \n",
       "is_iceberg_2      0.995483  \n",
       "is_iceberg_3      0.986247  \n",
       "is_iceberg_4      0.979320  \n",
       "is_iceberg_5      0.990099  \n",
       "is_iceberg_6      0.991532  \n",
       "is_iceberg_7      0.988391  \n",
       "is_iceberg_8      1.000000  "
      ]
     },
     "execution_count": 3,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "# check correlation\n",
    "concat_sub.corr()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 4,
   "metadata": {},
   "outputs": [],
   "source": [
    "# get the data fields ready for stacking\n",
    "concat_sub['is_iceberg_max'] = concat_sub.iloc[:, 1:10].max(axis=1)\n",
    "concat_sub['is_iceberg_min'] = concat_sub.iloc[:, 1:10].min(axis=1)\n",
    "concat_sub['is_iceberg_mean'] = concat_sub.iloc[:, 1:10].mean(axis=1)\n",
    "concat_sub['is_iceberg_median'] = concat_sub.iloc[:, 1:10].median(axis=1)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 5,
   "metadata": {},
   "outputs": [],
   "source": [
    "# set up cutoff threshold for lower and upper bounds, easy to twist \n",
    "cutoff_lo = 0.8\n",
    "cutoff_hi = 0.2"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 6,
   "metadata": {},
   "outputs": [],
   "source": [
    "concat_sub['is_iceberg'] = concat_sub['is_iceberg_median']\n",
    "concat_sub[['id','is_iceberg']].to_csv('./result/ensemble/stack_median_final98.csv', \n",
    "                                        index=False, float_format='%.6f')"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 7,
   "metadata": {},
   "outputs": [],
   "source": [
    "concat_sub['is_iceberg'] = np.where(np.all(concat_sub.iloc[:,1:10] > cutoff_lo, axis=1), \n",
    "                                    concat_sub['is_iceberg_max'], \n",
    "                                    np.where(np.all(concat_sub.iloc[:,1:10] < cutoff_hi, axis=1),\n",
    "                                             concat_sub['is_iceberg_min'], \n",
    "                                             concat_sub['is_iceberg_median']))\n",
    "concat_sub[['id', 'is_iceberg']].to_csv('./result/ensemble/stack_minmax_median_final98.csv', \n",
    "                                        index=False, float_format='%.6f')"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 8,
   "metadata": {},
   "outputs": [],
   "source": [
    "concat_sub['is_iceberg'] = np.where(np.all(concat_sub.iloc[:,1:10] > cutoff_lo, axis=1), \n",
    "                                    concat_sub['is_iceberg_max'], \n",
    "                                    np.where(np.all(concat_sub.iloc[:,1:10] < cutoff_hi, axis=1),\n",
    "                                             concat_sub['is_iceberg_min'], \n",
    "                                             concat_sub['is_iceberg_mean']))\n",
    "concat_sub[['id', 'is_iceberg']].to_csv('./result/ensemble/stack_minmax_mean_final98.csv', \n",
    "                                        index=False, float_format='%.6f')"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 15,
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "0.16576547509689682\n",
      "0.15747314142725255\n",
      "0.15762275440113052\n"
     ]
    }
   ],
   "source": [
    "stack_median = pd.read_csv('./result/ensemble/stack_median_final98.csv')\n",
    "stack_minmax = pd.read_csv('./result/ensemble/stack_minmax_median_final98.csv')\n",
    "stack_minmax2 = pd.read_csv('./result/ensemble/stack_minmax_mean_final98.csv')\n",
    "\n",
    "from sklearn.metrics import log_loss\n",
    "print(log_loss(y_test, stack_median['is_iceberg']))\n",
    "print(log_loss(y_test, stack_minmax['is_iceberg']))\n",
    "print(log_loss(y_test, stack_minmax2['is_iceberg']))"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Model Selection"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 115,
   "metadata": {},
   "outputs": [],
   "source": [
    "model1 = pd.read_csv('./result/ensemble/stack_minmax_median91.csv')\n",
    "model2 = pd.read_csv('./result/ensemble/stack_median96.csv')\n",
    "model3 = pd.read_csv('./result/ensemble/stack_minmax_median97.csv')"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 178,
   "metadata": {},
   "outputs": [],
   "source": [
    "num1 = 0.90\n",
    "num2 = 0.05\n",
    "num3 = 0.05"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 179,
   "metadata": {},
   "outputs": [],
   "source": [
    "model4 = model1['is_iceberg'] * num1 + model2['is_iceberg'] * num2 + model3['is_iceberg'] * num3"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 180,
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "0.1686667011544887\n"
     ]
    }
   ],
   "source": [
    "print(log_loss(y_test, model4))"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": []
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python [conda env:tensorflow]",
   "language": "python",
   "name": "conda-env-tensorflow-py"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.5.5"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 2
}
